In [ ]:
# Installing required libraries
!pip install pandas numpy nltk textblob gensim matplotlib seaborn wordcloud scikit-learn

# Importing libraries
import pandas as pd
import numpy as np
import nltk
from textblob import TextBlob
from gensim import corpora
from gensim.models import LdaModel
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA_sklearn
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 71.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Cinnamon Grand Colombo

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()  # Upload file named e.g., "1. CinnamonGrandColombo-TripAdvisor.xlsx"

Saving 1. CinnamonGrandColombo-TripAdvisor.xlsx to 1. CinnamonGrandColombo-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '1. CinnamonGrandColombo-TripAdvisor.xlsx'

# Load Excel file
CinnGrand_Colombo_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'CinnGrand_Colombo_TA'")
print("Shape (Rows, Columns):", CinnGrand_Colombo_TA.shape)

TripAdvisor file loaded successfully as 'CinnGrand_Colombo_TA'
Shape (Rows, Columns): (2500, 185)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(CinnGrand_Colombo_TA.columns.tolist())

print("\nData types and info:")
CinnGrand_Colombo_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = CinnGrand_Colombo_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(CinnGrand_Colombo_TA) * 100).round(2))

Missing values per column:
ownerResponse                        2500
ownerResponse/connectionToSubject      69
ownerResponse/id                       68
ownerResponse/lang                     68
ownerResponse/publishedDate            68
                                     ... 
user/userLocation                    2500
user/userLocation/id                 1783
user/userLocation/name               1783
user/userLocation/shortName          1783
user/username                           3
Length: 155, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            71.32
user/userLocation/name          71.32
user/userLocation/shortName     71.32
user/username                    0.12
Length: 185, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = CinnGrand_Colombo_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(CinnGrand_Colombo_TA[CinnGrand_Colombo_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in CinnGrand_Colombo_TA.columns:
    print(f"{col}: {CinnGrand_Colombo_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 3 unique values
id: 2500 unique values
lang: 17 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 4 unique values
ownerResponse/id: 2432 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 380 unique values
ownerResponse/responder: 8 unique values
ownerResponse/text: 1934 unique values
photos/0/height: 97 unique values
photos/0/id: 520 unique values
photos/0/image: 520 unique values
photos/0/width: 113 unique values
photos/1/height: 58 unique values
photos/1/id: 217 unique values
photos/1/image: 217 unique values
photos/1/width: 69 unique values
photos/2/height: 44 unique values
photos/2/id: 122 unique values
photos/2/image: 122 unique values
photos/2/width: 45 unique values
photos/3/height: 30 unique values
photos/3/id: 75 unique values
photos/3/image: 75 unique values
photos/3/width: 31 unique values
photos/4/height: 24 unique values
photos/4/id: 44 uniqu

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in CinnGrand_Colombo_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = CinnGrand_Colombo_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in CinnGrand_Colombo_TA.columns if col.startswith('subratings/')]
CinnGrand_Colombo_TA = pd.concat([CinnGrand_Colombo_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
CinnGrand_Colombo_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform

In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(CinnGrand_Colombo_TA.columns.tolist())
print(f"\nTotal columns: {len(CinnGrand_Colombo_TA.columns)} | Total rows: {len(CinnGrand_Colombo_TA)}")

# Dataset info
print("\nDataset Info:")
CinnGrand_Colombo_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(CinnGrand_Colombo_TA[sub_rating_cols].head(10))

Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,5.0,5.0,NaN,4.0,5.0,NaN,TripAdvisor
1,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,5.0,5.0,NaN,5.0,5.0,NaN,TripAdvisor
5,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
7,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
CinnGrand_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Columns: 180 entries, helpfulVotes to platform
dtypes: float64(109), int64(12), object(59)
memory usage: 3.4+ MB


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',
    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
CinnGrand_Colombo_TA = CinnGrand_Colombo_TA.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
CinnGrand_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   2500 non-null   object 
 1   ownerResponse/publishedDate            2432 non-null   object 
 2   ownerResponse/text                     2432 non-null   object 
 3   placeInfo/name                         2500 non-null   object 
 4   placeInfo/rating                       2500 non-null   float64
 5   publishedDate                          2500 non-null   object 
 6   publishedPlatform                      2500 non-null   object 
 7   rating                                 2500 non-null   int64  
 8   roomTip                                195 non-null    object 
 9   text                                   2500 non-null   object 
 10  title                                  2491 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
CinnGrand_Colombo_TA = CinnGrand_Colombo_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})

In [ ]:
CinnGrand_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    2500 non-null   object 
 1   OwnerResponsepublishedDate  2432 non-null   object 
 2   OwnerResponse               2432 non-null   object 
 3   Hotel/Resort                2500 non-null   object 
 4   placeInfo/rating            2500 non-null   float64
 5   RatingPublishedDate         2500 non-null   object 
 6   RatingPublishedPlatform     2500 non-null   object 
 7   HotelRating                 2500 non-null   int64  
 8   RoomtipText                 195 non-null    object 
 9   ReviewText                  2500 non-null   object 
 10  ReviewTitle                 2491 non-null   object 
 11  TripType                    2500 non-null   object 
 12  TotalReviewsbyUser          2497 non-null   float64
 13  UserName                    2497 

In [ ]:
display(CinnGrand_Colombo_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,"I'll start with the food - it was excellent, e...",...,Aruni ushani P,NaN,NaN,5.0,5.0,NaN,4.0,NaN,5.0,TripAdvisor
1,en,2024-09-26,"Dear Mary, \n\nThank you for your wonderful fe...",Cinnamon Grand Colombo,4.8,2024-09-26,OTHER,5,NaN,Beautiful hotel. Extremely clean and tidy. Ver...,...,596mary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,It was very crowded since the following day wa...,...,Duleep L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,Excellent service from Ms Aveesha. Very helpfu...,...,Global40281373510,"Subang Jaya, Malaysia",Subang Jaya,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,That's a great experience to be honest it's pr...,...,Sumathi S,NaN,NaN,5.0,5.0,NaN,5.0,NaN,5.0,TripAdvisor
5,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,Indisa was amazing handling our order. \nfull ...,...,Pasan R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,Indisa was kind and provided us with a good se...,...,Chathuranga Warakapitiya,"Sri Jayawardenepura, Sri Lanka",Sri Jayawardenepura,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
7,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,Milan is a very Good staff member very kind an...,...,Teddy x,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,Jayaweera served us for the night. It was a me...,...,Sheryl R,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,en,NaN,NaN,Cinnamon Grand Colombo,4.8,2025-12-03,OTHER,5,NaN,I recently had the opportunity to dine at the...,...,shalini R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'CinnGrand_Colombo_TA_Cleaned.xlsx'

CinnGrand_Colombo_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as CinnGrand_Colombo_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cinnamon Lakeside Colombo

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 2. CinnamonLakesideColombo-TripAdvisor.xlsx to 2. CinnamonLakesideColombo-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '2. CinnamonLakesideColombo-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
CinnLakeside_Colombo_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'CinnLakeside_Colombo_TA'")
print("Shape (Rows, Columns):", CinnLakeside_Colombo_TA.shape)


TripAdvisor file loaded successfully as 'CinnLakeside_Colombo_TA'
Shape (Rows, Columns): (2500, 245)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(CinnLakeside_Colombo_TA.columns.tolist())

print("\nData types and info:")
CinnLakeside_Colombo_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = CinnLakeside_Colombo_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(CinnLakeside_Colombo_TA) * 100).round(2))

Missing values per column:
ownerResponse                        2500
ownerResponse/connectionToSubject       9
ownerResponse/id                        9
ownerResponse/lang                      9
ownerResponse/publishedDate             9
                                     ... 
user/userLocation                    2500
user/userLocation/id                 1864
user/userLocation/name               1864
user/userLocation/shortName          1864
user/username                           1
Length: 217, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            74.56
user/userLocation/name          74.56
user/userLocation/shortName     74.56
user/username                    0.04
Length: 245, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = CinnLakeside_Colombo_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(CinnLakeside_Colombo_TA[CinnLakeside_Colombo_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in CinnLakeside_Colombo_TA.columns:
    print(f"{col}: {CinnLakeside_Colombo_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 5 unique values
id: 2500 unique values
lang: 10 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 1 unique values
ownerResponse/id: 2491 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 745 unique values
ownerResponse/responder: 3 unique values
ownerResponse/text: 2169 unique values
photos/0/height: 123 unique values
photos/0/id: 405 unique values
photos/0/image: 405 unique values
photos/0/width: 120 unique values
photos/1/height: 63 unique values
photos/1/id: 177 unique values
photos/1/image: 177 unique values
photos/1/width: 60 unique values
photos/2/height: 36 unique values
photos/2/id: 116 unique values
photos/2/image: 116 unique values
photos/2/width: 40 unique values
photos/3/height: 28 unique values
photos/3/id: 66 unique values
photos/3/image: 66 unique values
photos/3/width: 32 unique values
photos/4/height: 22 unique values
photos/4/id: 47 uniq

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in CinnLakeside_Colombo_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = CinnLakeside_Colombo_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in CinnLakeside_Colombo_TA.columns if col.startswith('subratings/')]
CinnLakeside_Colombo_TA = pd.concat([CinnLakeside_Colombo_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
CinnLakeside_Colombo_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform


In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(CinnLakeside_Colombo_TA.columns.tolist())
print(f"\nTotal columns: {len(CinnLakeside_Colombo_TA.columns)} | Total rows: {len(CinnLakeside_Colombo_TA)}")

# Dataset info
print("\nDataset Info:")
CinnLakeside_Colombo_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(CinnLakeside_Colombo_TA[sub_rating_cols].head(10))


Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
1,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,5.0,5.0,3.0,5.0,4.0,5.0,TripAdvisor
4,5.0,5.0,4.0,5.0,4.0,5.0,TripAdvisor
5,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
6,4.0,4.0,4.0,5.0,4.0,4.0,TripAdvisor
7,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,4.0,3.0,3.0,4.0,3.0,4.0,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',
    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',
    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',

    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',

    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',



    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',
    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',

    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',

    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',
    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
CinnLakeside_Colombo_TA = CinnLakeside_Colombo_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:

CinnLakeside_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   2500 non-null   object 
 1   ownerResponse/publishedDate            2491 non-null   object 
 2   ownerResponse/text                     2491 non-null   object 
 3   placeInfo/name                         2500 non-null   object 
 4   placeInfo/rating                       2500 non-null   float64
 5   publishedDate                          2500 non-null   object 
 6   publishedPlatform                      2500 non-null   object 
 7   rating                                 2500 non-null   int64  
 8   roomTip                                190 non-null    object 
 9   text                                   2500 non-null   object 
 10  title                                  2498 non-null   object 
 11  trip

In [ ]:

# Re-naming/Standardizing the columns
CinnLakeside_Colombo_TA = CinnLakeside_Colombo_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})

In [ ]:

CinnLakeside_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    2500 non-null   object 
 1   OwnerResponsepublishedDate  2491 non-null   object 
 2   OwnerResponse               2491 non-null   object 
 3   Hotel/Resort                2500 non-null   object 
 4   placeInfo/rating            2500 non-null   float64
 5   RatingPublishedDate         2500 non-null   object 
 6   RatingPublishedPlatform     2500 non-null   object 
 7   HotelRating                 2500 non-null   int64  
 8   RoomtipText                 190 non-null    object 
 9   ReviewText                  2500 non-null   object 
 10  ReviewTitle                 2498 non-null   object 
 11  TripType                    2499 non-null   object 
 12  TotalReviewsbyUser          2499 non-null   float64
 13  UserName                    2499 

In [ ]:

display(CinnLakeside_Colombo_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Cinnamon Lakeside Colombo,4.7,2025-12-10,OTHER,5,NaN,"Comfortable stay, awesome room view from sun k...",...,Safari42190771450,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
1,en,2024-09-29,"Dear Guest,\n\nGreetings from Cinnamon Lakesid...",Cinnamon Lakeside Colombo,4.7,2024-09-29,MOBILE,5,This is a great hotel to explore Colombo. Don’...,Our stay at the **Cinnamon Lakeside Hotel** in...,...,145piyushs,India,India,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,en,NaN,NaN,Cinnamon Lakeside Colombo,4.7,2025-12-10,MOBILE,5,NaN,We came to the cinnamon lakeside late night 12...,...,Isuru L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,en,NaN,NaN,Cinnamon Lakeside Colombo,4.7,2025-12-10,OTHER,5,NaN,"Great dining experience, nice calm atmosphere,...",...,Anupa S,NaN,NaN,5.0,5.0,3.0,5.0,5.0,4.0,TripAdvisor
4,en,NaN,NaN,Cinnamon Lakeside Colombo,4.7,2025-12-09,OTHER,5,NaN,Location is great. Easy access to places. Pret...,...,Ramani S,NaN,NaN,5.0,5.0,4.0,5.0,5.0,4.0,TripAdvisor
5,en,2025-12-08,"Dear Guest,\n\nThank you for your wonderful fe...",Cinnamon Lakeside Colombo,4.7,2025-12-08,OTHER,5,NaN,Service was excellent topnotch.Staff are very ...,...,fnm1227,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
6,en,2025-12-05,"Dear Guest,\nThank you so much for your kind w...",Cinnamon Lakeside Colombo,4.7,2025-12-02,OTHER,5,NaN,The staff were excellent. Their genuine smile...,...,May W,"Sydney, Australia",Sydney,4.0,4.0,4.0,5.0,4.0,4.0,TripAdvisor
7,en,2025-12-02,"Dear Manoj,\nThank you for your kind feedback!...",Cinnamon Lakeside Colombo,4.7,2025-12-01,OTHER,5,NaN,Had a comfortable stay at the hotel and met my...,...,129manoj,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,en,2025-12-02,"Dear Guest,\nThank you so much for your kind w...",Cinnamon Lakeside Colombo,4.7,2025-11-28,OTHER,5,NaN,Had a wonderful dinner buffet with exceptional...,...,Ali Mowjood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,en,2025-12-02,"Dear Guest,\nThank you for your feedback and f...",Cinnamon Lakeside Colombo,4.7,2025-11-27,OTHER,4,NaN,"Helpfull Staff and good service, but be aware ...",...,ulfmat,"Hammerfest, Norway",Hammerfest,4.0,3.0,3.0,4.0,4.0,3.0,TripAdvisor


In [ ]:

# Save the cleaned DataFrame as Excel
excel_filename = 'CinnLakeside_Colombo_TA_Cleaned.xlsx'

CinnLakeside_Colombo_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)


Dataset saved as CinnLakeside_Colombo_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cinnamon Red Colombo

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 3. CinnamonRedColombo-TripAdvisor.xlsx to 3. CinnamonRedColombo-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '3. CinnamonRedColombo-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
CinnRed_Colombo_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'CinnRed_Colombo_TA'")
print("Shape (Rows, Columns):", CinnRed_Colombo_TA.shape)

TripAdvisor file loaded successfully as 'CinnRed_Colombo_TA'
Shape (Rows, Columns): (2500, 221)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(CinnRed_Colombo_TA.columns.tolist())

print("\nData types and info:")
CinnRed_Colombo_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = CinnRed_Colombo_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(CinnRed_Colombo_TA) * 100).round(2))

Missing values per column:
ownerResponse                        2500
ownerResponse/connectionToSubject      22
ownerResponse/id                       22
ownerResponse/lang                     22
ownerResponse/publishedDate            22
                                     ... 
user/userLocation                    2500
user/userLocation/id                 1350
user/userLocation/name               1350
user/userLocation/shortName          1350
user/username                           6
Length: 193, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            54.00
user/userLocation/name          54.00
user/userLocation/shortName     54.00
user/username                    0.24
Length: 221, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = CinnRed_Colombo_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(CinnRed_Colombo_TA[CinnRed_Colombo_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:

# Unique values per column
print("Unique values count per column:")
for col in CinnRed_Colombo_TA.columns:
    print(f"{col}: {CinnRed_Colombo_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 8 unique values
id: 2500 unique values
lang: 21 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 1 unique values
ownerResponse/id: 2478 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 856 unique values
ownerResponse/responder: 3 unique values
ownerResponse/text: 1694 unique values
photos/0/height: 127 unique values
photos/0/id: 522 unique values
photos/0/image: 522 unique values
photos/0/width: 124 unique values
photos/1/height: 92 unique values
photos/1/id: 310 unique values
photos/1/image: 310 unique values
photos/1/width: 87 unique values
photos/2/height: 67 unique values
photos/2/id: 201 unique values
photos/2/image: 201 unique values
photos/2/width: 61 unique values
photos/3/height: 47 unique values
photos/3/id: 130 unique values
photos/3/image: 130 unique values
photos/3/width: 42 unique values
photos/4/height: 34 unique values
photos/4/id: 81 un

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in CinnRed_Colombo_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = CinnRed_Colombo_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in CinnRed_Colombo_TA.columns if col.startswith('subratings/')]
CinnRed_Colombo_TA = pd.concat([CinnRed_Colombo_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
CinnRed_Colombo_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform


In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(CinnRed_Colombo_TA.columns.tolist())
print(f"\nTotal columns: {len(CinnRed_Colombo_TA.columns)} | Total rows: {len(CinnRed_Colombo_TA)}")

# Dataset info
print("\nDataset Info:")
CinnRed_Colombo_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(CinnRed_Colombo_TA[sub_rating_cols].head(10))

Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,4.0,5.0,5.0,5.0,5.0,3.0,TripAdvisor
3,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
4,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,5.0,5.0,5.0,5.0,5.0,NaN,TripAdvisor
6,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
7,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,5.0,4.0,5.0,4.0,4.0,5.0,TripAdvisor
9,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',
    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',

    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
CinnRed_Colombo_TA = CinnRed_Colombo_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
CinnRed_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   2500 non-null   object 
 1   ownerResponse/publishedDate            2478 non-null   object 
 2   ownerResponse/text                     2478 non-null   object 
 3   placeInfo/name                         2500 non-null   object 
 4   placeInfo/rating                       2500 non-null   float64
 5   publishedDate                          2500 non-null   object 
 6   publishedPlatform                      2500 non-null   object 
 7   rating                                 2500 non-null   int64  
 8   roomTip                                336 non-null    object 
 9   text                                   2500 non-null   object 
 10  title                                  2494 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
CinnRed_Colombo_TA = CinnRed_Colombo_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})

In [ ]:
CinnRed_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    2500 non-null   object 
 1   OwnerResponsepublishedDate  2478 non-null   object 
 2   OwnerResponse               2478 non-null   object 
 3   Hotel/Resort                2500 non-null   object 
 4   placeInfo/rating            2500 non-null   float64
 5   RatingPublishedDate         2500 non-null   object 
 6   RatingPublishedPlatform     2500 non-null   object 
 7   HotelRating                 2500 non-null   int64  
 8   RoomtipText                 336 non-null    object 
 9   ReviewText                  2500 non-null   object 
 10  ReviewTitle                 2494 non-null   object 
 11  TripType                    2499 non-null   object 
 12  TotalReviewsbyUser          2494 non-null   float64
 13  UserName                    2493 

In [ ]:
display(CinnRed_Colombo_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Cinnamon Red Colombo,4.5,2025-12-03,OTHER,5,NaN,Wonderful and clean rooms! Shout out to Pathum...,...,Discover09198590313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,en,2024-10-01,"Dear Dav, \n\nThank you for choosing Cinnamon ...",Cinnamon Red Colombo,4.5,2024-09-26,MOBILE,5,NaN,Outstanding Hospitality at Cinnamon Red\n\nI h...,...,Dav M,"Northwood, United Kingdom",Northwood,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,en,NaN,NaN,Cinnamon Red Colombo,4.5,2025-12-03,OTHER,5,"Hace frío en las noches en el hotel, a veces h...","Room voy Gayan, 20th floor, soooo nice, friend...",...,Fer Chamorro,NaN,NaN,4.0,5.0,5.0,5.0,3.0,5.0,TripAdvisor
3,en,NaN,NaN,Cinnamon Red Colombo,4.5,2025-12-02,OTHER,5,NaN,I had a great stay at Cinnamon Red Colombo. Th...,...,Sumedha U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
4,en,2025-12-01,"Dear Guest,\nThank you for your review followi...",Cinnamon Red Colombo,4.5,2025-12-01,OTHER,5,NaN,I am stay on 1210.Room is very cleaned and com...,...,Leslie G,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,en,2025-12-01,"Dear Guest,\nThank you for your review followi...",Cinnamon Red Colombo,4.5,2025-12-01,OTHER,5,NaN,Gorgeous hotel with a rooftop bar and pool. Br...,...,Sarah C,NaN,NaN,5.0,5.0,5.0,5.0,NaN,5.0,TripAdvisor
6,en,2025-12-01,"Dear Guest,\nThank you for your review followi...",Cinnamon Red Colombo,4.5,2025-11-30,OTHER,5,NaN,Mahesh did a great job in cleaning up our room...,...,Colin F,"Sydney, Australia",Sydney,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
7,zhCN,2025-12-01,"Dear Guest,\nThank you for your review followi...",Cinnamon Red Colombo,4.5,2025-11-30,OTHER,5,NaN,酒店视野很好，服务员也很nice，特别是Mahesh，很有礼貌，会提前沟通确定打扫房间的时间...,...,nicolas z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,es,2025-12-01,"Dear Guest,\nThank you for your review followi...",Cinnamon Red Colombo,4.5,2025-11-29,OTHER,4,NaN,"El hotel es moderno, tiene unas vistas especta...",...,Patri P,NaN,NaN,5.0,4.0,5.0,4.0,5.0,4.0,TripAdvisor
9,en,2025-12-01,"Dear Guest,\nThank you for your review followi...",Cinnamon Red Colombo,4.5,2025-11-27,MOBILE,5,NaN,Mahesh was a wonderful trustworthy housekeeper...,...,majorcraique49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'CinnRed_Colombo_TA_Cleaned.xlsx'

CinnRed_Colombo_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as CinnRed_Colombo_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cinnamon Life at City of Dreams Colombo

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 4. CinnamonLifeatCityofDreams-TripAdvisor.xlsx to 4. CinnamonLifeatCityofDreams-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '4. CinnamonLifeatCityofDreams-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
CinnLife_Colombo_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'CinnLife_Colombo_TA'")
print("Shape (Rows, Columns):", CinnLife_Colombo_TA.shape)


TripAdvisor file loaded successfully as 'CinnLife_Colombo_TA'
Shape (Rows, Columns): (1221, 213)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(CinnLife_Colombo_TA.columns.tolist())

print("\nData types and info:")
CinnLife_Colombo_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = CinnLife_Colombo_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(CinnLife_Colombo_TA) * 100).round(2))

Missing values per column:
ownerResponse                        1221
ownerResponse/connectionToSubject      17
ownerResponse/id                       17
ownerResponse/lang                     17
ownerResponse/publishedDate            17
                                     ... 
user/userLocation                    1221
user/userLocation/id                  920
user/userLocation/name                920
user/userLocation/shortName           920
user/username                           1
Length: 183, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            75.35
user/userLocation/name          75.35
user/userLocation/shortName     75.35
user/username                    0.08
Length: 213, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = CinnLife_Colombo_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(CinnLife_Colombo_TA[CinnLife_Colombo_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in CinnLife_Colombo_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = CinnLife_Colombo_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in CinnLife_Colombo_TA.columns if col.startswith('subratings/')]
CinnLife_Colombo_TA = pd.concat([CinnLife_Colombo_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
CinnLife_Colombo_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform


In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(CinnLife_Colombo_TA.columns.tolist())
print(f"\nTotal columns: {len(CinnLife_Colombo_TA.columns)} | Total rows: {len(CinnGrand_Colombo_TA)}")

# Dataset info
print("\nDataset Info:")
CinnLife_Colombo_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(CinnLife_Colombo_TA[sub_rating_cols].head(10))

Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,5.0,4.0,5.0,3.0,2.0,1.0,TripAdvisor
1,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
5,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
6,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
7,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
CinnLife_Colombo_TA = CinnLife_Colombo_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
CinnLife_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   1221 non-null   object 
 1   ownerResponse/publishedDate            1204 non-null   object 
 2   ownerResponse/text                     1204 non-null   object 
 3   placeInfo/name                         1221 non-null   object 
 4   placeInfo/rating                       1221 non-null   float64
 5   publishedDate                          1221 non-null   object 
 6   publishedPlatform                      1221 non-null   object 
 7   rating                                 1221 non-null   int64  
 8   roomTip                                114 non-null    object 
 9   text                                   1221 non-null   object 
 10  title                                  1202 non-null   object 
 11  trip

In [ ]:

# Re-naming/Standardizing the columns
CinnLife_Colombo_TA = CinnLife_Colombo_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})

In [ ]:

CinnLife_Colombo_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    1221 non-null   object 
 1   OwnerResponsepublishedDate  1204 non-null   object 
 2   OwnerResponse               1204 non-null   object 
 3   Hotel/Resort                1221 non-null   object 
 4   placeInfo/rating            1221 non-null   float64
 5   RatingPublishedDate         1221 non-null   object 
 6   RatingPublishedPlatform     1221 non-null   object 
 7   HotelRating                 1221 non-null   int64  
 8   RoomtipText                 114 non-null    object 
 9   ReviewText                  1221 non-null   object 
 10  ReviewTitle                 1202 non-null   object 
 11  TripType                    1221 non-null   object 
 12  TotalReviewsbyUser          1220 non-null   float64
 13  UserName                    1220 

In [ ]:

display(CinnLife_Colombo_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-06,OTHER,3,NaN,Absolutely horrible food. Everything tastes ve...,...,Anusha S,NaN,NaN,5.0,4.0,5.0,3.0,1.0,2.0,TripAdvisor
1,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-06,MOBILE,2,NaN,Our check-in experience was unfortunately disa...,...,BSManiam,"Singapore, Singapore",Singapore,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-06,OTHER,4,NaN,Very welcoming and friendly staff. Peaceful at...,...,House o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-05,OTHER,5,NaN,Amazing stay.. we loved eveything. Food is lov...,...,Sriya H,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-05,OTHER,5,NaN,kids are very happy to play here. they have l...,...,Lu H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
5,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-05,OTHER,5,NaN,\nHotel experience was very good and comfortab...,...,sidathsd69,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
6,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-05,OTHER,5,NaN,Visited the hotel yesterday evening and had a ...,...,sudasna s,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
7,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-05,OTHER,5,NaN,We were placed into rooms at Cinnamon Life aft...,...,Anuki D,"Colombo, Sri Lanka",Colombo,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-05,OTHER,5,NaN,I recently went to City of Dreams for a press ...,...,Bashini R,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,en,NaN,NaN,Cinnamon Life City of Dreams,4.8,2025-12-04,OTHER,5,NaN,Amazing experience from this hotel and good st...,...,Ravinder K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'CinnLife_Colombo_TA_Cleaned.xlsx'

CinnLife_Colombo_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as CinnLife_Colombo_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Kandy Myst by Cinnamon

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 5. KandyMystbyCinnamon-Booking.csv to 5. KandyMystbyCinnamon-Booking.csv


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '5. KandyMystbyCinnamon-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
Cinn_KandyMyst_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'Cinn_KandyMyst_TA'")
print("Shape (Rows, Columns):", Cinn_KandyMyst_TA.shape)


TripAdvisor file loaded successfully as 'Cinn_KandyMyst_TA'
Shape (Rows, Columns): (379, 256)


In [ ]:

# Printing a list of all the columns in the dataset
print("All columns:")
print(Cinn_KandyMyst_TA.columns.tolist())

print("\nData types and info:")
Cinn_KandyMyst_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = Cinn_KandyMyst_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(Cinn_KandyMyst_TA) * 100).round(2))

Missing values per column:
ownerResponse                        379
ownerResponse/connectionToSubject      8
ownerResponse/id                       8
ownerResponse/lang                     8
ownerResponse/publishedDate            8
                                    ... 
user/avatar/width                     16
user/userLocation                    379
user/userLocation/id                 279
user/userLocation/name               279
user/userLocation/shortName          279
Length: 222, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            73.61
user/userLocation/name          73.61
user/userLocation/shortName     73.61
user/username                    0.00
Length: 256, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = Cinn_KandyMyst_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(Cinn_KandyMyst_TA[Cinn_KandyMyst_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in Cinn_KandyMyst_TA.columns:
    print(f"{col}: {Cinn_KandyMyst_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 2 unique values
id: 379 unique values
lang: 10 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 1 unique values
ownerResponse/id: 371 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 83 unique values
ownerResponse/responder: 3 unique values
ownerResponse/text: 371 unique values
photos/0/height: 43 unique values
photos/0/id: 128 unique values
photos/0/image: 128 unique values
photos/0/width: 45 unique values
photos/1/height: 29 unique values
photos/1/id: 73 unique values
photos/1/image: 73 unique values
photos/1/width: 33 unique values
photos/2/height: 23 unique values
photos/2/id: 49 unique values
photos/2/image: 49 unique values
photos/2/width: 26 unique values
photos/3/height: 20 unique values
photos/3/id: 31 unique values
photos/3/image: 31 unique values
photos/3/width: 21 unique values
photos/4/height: 12 unique values
photos/4/id: 19 unique values


In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in Cinn_KandyMyst_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = Cinn_KandyMyst_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in Cinn_KandyMyst_TA.columns if col.startswith('subratings/')]
Cinn_KandyMyst_TA = pd.concat([Cinn_KandyMyst_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
Cinn_KandyMyst_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform



In [ ]:

# Show updated columns
print("Updated columns after sub-rating extraction:")
print(Cinn_KandyMyst_TA.columns.tolist())
print(f"\nTotal columns: {len(Cinn_KandyMyst_TA.columns)} | Total rows: {len(CinnGrand_Colombo_TA)}")

# Dataset info
print("\nDataset Info:")
Cinn_KandyMyst_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(Cinn_KandyMyst_TA[sub_rating_cols].head(10))

Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,NaN,5.0,5.0,NaN,5.0,NaN,TripAdvisor
6,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
7,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,5.0,NaN,5.0,5.0,NaN,5.0,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',
    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',

    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',
    'photos/44/height',
    'photos/44/id',
    'photos/44/image',
    'photos/44/width',
    'photos/45/height',
    'photos/45/id',
    'photos/45/image',
    'photos/45/width',
    'photos/46/height',
    'photos/46/id',
    'photos/46/image',
    'photos/46/width',

    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
Cinn_KandyMyst_TA = Cinn_KandyMyst_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
Cinn_KandyMyst_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   379 non-null    object 
 1   ownerResponse/publishedDate            371 non-null    object 
 2   ownerResponse/text                     371 non-null    object 
 3   placeInfo/name                         379 non-null    object 
 4   placeInfo/rating                       379 non-null    float64
 5   publishedDate                          379 non-null    object 
 6   publishedPlatform                      379 non-null    object 
 7   rating                                 379 non-null    int64  
 8   roomTip                                48 non-null     object 
 9   text                                   379 non-null    object 
 10  title                                  378 non-null    object 
 11  tripTy

In [ ]:
# Re-naming/Standardizing the columns
Cinn_KandyMyst_TA = Cinn_KandyMyst_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})


In [ ]:
Cinn_KandyMyst_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    379 non-null    object 
 1   OwnerResponsepublishedDate  371 non-null    object 
 2   OwnerResponse               371 non-null    object 
 3   Hotel/Resort                379 non-null    object 
 4   placeInfo/rating            379 non-null    float64
 5   RatingPublishedDate         379 non-null    object 
 6   RatingPublishedPlatform     379 non-null    object 
 7   HotelRating                 379 non-null    int64  
 8   RoomtipText                 48 non-null     object 
 9   ReviewText                  379 non-null    object 
 10  ReviewTitle                 378 non-null    object 
 11  TripType                    378 non-null    object 
 12  TotalReviewsbyUser          379 non-null    int64  
 13  UserName                    379 non

In [ ]:
display(Cinn_KandyMyst_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,sk,NaN,NaN,Kandy Myst By Cinnamon,4.8,2025-12-01,MOBILE,5,NaN,Veľmi pekný hotel. Nie celkom v centre Kandy. ...,...,Slavomíra R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,en,2025-10-22,"Dear Raymond,\n\nGreetings from Kandy Myst by ...",Kandy Myst By Cinnamon,4.8,2025-10-20,MOBILE,5,NaN,"Excellent hotel with well appointed, spotlessl...",...,GogoCebuCity,"Ho Chi Minh City, Vietnam",Ho Chi Minh City,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,en,2025-11-29,"Dear Yanick,\n\nGreetings from Kandy Myst by C...",Kandy Myst By Cinnamon,4.8,2025-11-28,OTHER,5,NaN,We are writing on behalf of the World Universi...,...,Yanick D,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,en,2025-11-25,"Dear Anushika,\n\nGreetings from Kandy Myst by...",Kandy Myst By Cinnamon,4.8,2025-11-25,OTHER,5,NaN,We came to kandy myst hotel for celebrate my b...,...,Anushika M,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,en,2025-11-29,"Dear Luciaz,\n\nGreetings from Kandy Myst by C...",Kandy Myst By Cinnamon,4.8,2025-11-24,OTHER,5,NaN,We originally planned to stay just one night a...,...,Lucia Z,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,en,2025-11-29,"Dear Raweeng,\n\nGreetings from Kandy Myst by ...",Kandy Myst By Cinnamon,4.8,2025-11-23,OTHER,5,Highly rwcommended.\nSuranga who working @ fro...,Nice and very frindly place to say. Suranga wh...,...,Raween G,NaN,NaN,NaN,5.0,5.0,NaN,NaN,5.0,TripAdvisor
6,en,2025-11-29,"Dear Dinithi,\n\nGreetings from Kandy Myst by ...",Kandy Myst By Cinnamon,4.8,2025-11-23,OTHER,5,NaN,Good friendly staff and delicious foods.hotel ...,...,Dinithi H,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
7,en,2025-11-29,"Dear pansilu,\n\nGreetings from Kandy Myst by ...",Kandy Myst By Cinnamon,4.8,2025-11-23,OTHER,5,NaN,Our family had an amazing experience at Kandy ...,...,Pansilu K,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,en,2025-11-29,"Dear kawaieay,\t\n\nGreetings from Kandy Myst ...",Kandy Myst By Cinnamon,4.8,2025-11-23,OTHER,5,NaN,Nice hotel clean excellent food and special se...,...,Ali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,en,2025-11-29,"Dear Saraha,\n\nGreetings from Kandy Myst by C...",Kandy Myst By Cinnamon,4.8,2025-11-23,OTHER,5,NaN,The hotel is very clean and the service is exc...,...,Sarah A,NaN,NaN,5.0,NaN,5.0,5.0,5.0,NaN,TripAdvisor


In [ ]:

# Save the cleaned DataFrame as Excel
excel_filename = 'Cinn_KandyMyst_TA_Cleaned.xlsx'

Cinn_KandyMyst_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as Cinn_KandyMyst_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cinnamon Bey Beruwala

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 6. CinnamonBeyBeruwala-Booking.csv to 6. CinnamonBeyBeruwala-Booking.csv


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '6. CinnamonBeyBeruwala-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
CinnBey_Beruwala_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'CinnBey_Beruwala_TA'")
print("Shape (Rows, Columns):", CinnBey_Beruwala_TA.shape)


TripAdvisor file loaded successfully as 'CinnBey_Beruwala_TA'
Shape (Rows, Columns): (1799, 261)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(CinnBey_Beruwala_TA.columns.tolist())

print("\nData types and info:")
CinnBey_Beruwala_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = CinnBey_Beruwala_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(CinnBey_Beruwala_TA) * 100).round(2))

Missing values per column:
ownerResponse                        1799
ownerResponse/connectionToSubject      10
ownerResponse/id                       10
ownerResponse/lang                     10
ownerResponse/publishedDate            10
                                     ... 
user/userLocation                    1799
user/userLocation/id                 1193
user/userLocation/name               1193
user/userLocation/shortName          1193
user/username                          10
Length: 235, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            66.31
user/userLocation/name          66.31
user/userLocation/shortName     66.31
user/username                    0.56
Length: 261, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = CinnBey_Beruwala_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(CinnBey_Beruwala_TA[CinnBey_Beruwala_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in CinnBey_Beruwala_TA.columns:
    print(f"{col}: {CinnBey_Beruwala_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 5 unique values
id: 1799 unique values
lang: 16 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 3 unique values
ownerResponse/id: 1789 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 439 unique values
ownerResponse/responder: 8 unique values
ownerResponse/text: 1442 unique values
photos/0/height: 112 unique values
photos/0/id: 535 unique values
photos/0/image: 535 unique values
photos/0/width: 120 unique values
photos/1/height: 91 unique values
photos/1/id: 398 unique values
photos/1/image: 398 unique values
photos/1/width: 95 unique values
photos/2/height: 78 unique values
photos/2/id: 326 unique values
photos/2/image: 326 unique values
photos/2/width: 88 unique values
photos/3/height: 64 unique values
photos/3/id: 258 unique values
photos/3/image: 257 unique values
photos/3/width: 71 unique values
photos/4/height: 55 unique values
photos/4/id: 208 u

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in CinnBey_Beruwala_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = CinnBey_Beruwala_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in CinnBey_Beruwala_TA.columns if col.startswith('subratings/')]
CinnBey_Beruwala_TA = pd.concat([CinnBey_Beruwala_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
CinnBey_Beruwala_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform


In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(CinnBey_Beruwala_TA.columns.tolist())
print(f"\nTotal columns: {len(CinnBey_Beruwala_TA.columns)} | Total rows: {len(CinnGrand_Colombo_TA)}")

# Dataset info
print("\nDataset Info:")
CinnBey_Beruwala_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(CinnBey_Beruwala_TA[sub_rating_cols].head(10))

Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
1,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,4.0,5.0,4.0,5.0,4.0,5.0,TripAdvisor
4,5.0,5.0,5.0,4.0,4.0,5.0,TripAdvisor
5,4.0,4.0,4.0,5.0,4.0,5.0,TripAdvisor
6,5.0,5.0,4.0,4.0,4.0,4.0,TripAdvisor
7,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',
    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',

    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',
    'photos/44/height',
    'photos/44/id',
    'photos/44/image',
    'photos/44/width',
    'photos/45/height',
    'photos/45/id',
    'photos/45/image',
    'photos/45/width',
    'photos/46/height',
    'photos/46/id',
    'photos/46/image',
    'photos/46/width',
    'photos/47/height',
    'photos/47/id',
    'photos/47/image',
    'photos/47/width',

    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
CinnBey_Beruwala_TA = CinnBey_Beruwala_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
CinnBey_Beruwala_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   1799 non-null   object 
 1   ownerResponse/publishedDate            1789 non-null   object 
 2   ownerResponse/text                     1789 non-null   object 
 3   placeInfo/name                         1799 non-null   object 
 4   placeInfo/rating                       1799 non-null   float64
 5   publishedDate                          1799 non-null   object 
 6   publishedPlatform                      1799 non-null   object 
 7   rating                                 1799 non-null   int64  
 8   roomTip                                229 non-null    object 
 9   text                                   1799 non-null   object 
 10  title                                  1789 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
CinnBey_Beruwala_TA = CinnBey_Beruwala_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})


In [ ]:
CinnBey_Beruwala_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    1799 non-null   object 
 1   OwnerResponsepublishedDate  1789 non-null   object 
 2   OwnerResponse               1789 non-null   object 
 3   Hotel/Resort                1799 non-null   object 
 4   placeInfo/rating            1799 non-null   float64
 5   RatingPublishedDate         1799 non-null   object 
 6   RatingPublishedPlatform     1799 non-null   object 
 7   HotelRating                 1799 non-null   int64  
 8   RoomtipText                 229 non-null    object 
 9   ReviewText                  1799 non-null   object 
 10  ReviewTitle                 1789 non-null   object 
 11  TripType                    1797 non-null   object 
 12  TotalReviewsbyUser          1789 non-null   float64
 13  UserName                    1789 

In [ ]:
display(CinnBey_Beruwala_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Cinnamon Bey Beruwala,4.4,2025-12-03,OTHER,5,NaN,The hotel gives the best experience with hospi...,...,AAKANKSHA S,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
1,en,2025-10-18,"Dear Valued Guest,\n\nGreetings from Cinnamon ...",Cinnamon Bey Beruwala,4.4,2025-10-17,MOBILE,5,NaN,The Cinnamon Bey hotel is luxurious and comfor...,...,Angelavert4,"Kingston-upon-Hull, United Kingdom",Kingston-upon-Hull,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,ru,NaN,NaN,Cinnamon Bey Beruwala,4.4,2025-12-03,OTHER,5,NaN,Отдыхали летом в низкий сезон. Никаких минусов...,...,Максим М,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,en,NaN,NaN,Cinnamon Bey Beruwala,4.4,2025-12-03,OTHER,5,NaN,Great experience. Amazing service by all staff...,...,Shriram K,NaN,NaN,4.0,5.0,4.0,5.0,5.0,4.0,TripAdvisor
4,ru,NaN,NaN,Cinnamon Bey Beruwala,4.4,2025-12-03,OTHER,4,NaN,Отдыхали на Шри-Ланке второй раз.\nОтель прекр...,...,Ирина М,NaN,NaN,5.0,5.0,5.0,4.0,5.0,4.0,TripAdvisor
5,en,NaN,NaN,Cinnamon Bey Beruwala,4.4,2025-12-02,OTHER,4,Towels for pool disappear early and get left o...,"My room boy Pathum was very kind, cleaned well...",...,NT64,"Leicestershire, United Kingdom",Leicestershire,4.0,4.0,4.0,5.0,5.0,4.0,TripAdvisor
6,ru,2025-12-01,"Dear guest,\n\nGreetings from Cinnamon Bey Ber...",Cinnamon Bey Beruwala,4.4,2025-12-01,OTHER,4,NaN,В отеле все понравилось но хотелось бы предупр...,...,Edgar S,NaN,NaN,5.0,5.0,4.0,4.0,4.0,4.0,TripAdvisor
7,en,2025-12-01,"Dear guest,\n\nGreetings from Cinnamon Bey Ber...",Cinnamon Bey Beruwala,4.4,2025-11-30,MOBILE,5,NaN,Here is a TripAdvisor-style 5-star review—clea...,...,Sally R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,en,2025-12-01,"Dear guest,\n\nGreetings from Cinnamon Bey Ber...",Cinnamon Bey Beruwala,4.4,2025-11-30,OTHER,5,NaN,Thank you very much. Stayed at the hotel for 4...,...,Ian F,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,en,2025-12-01,"Dear guest,\n\nGreetings from Cinnamon Bey Ber...",Cinnamon Bey Beruwala,4.4,2025-11-30,OTHER,5,NaN,Chirantha was super friendly and explained the...,...,Simran D,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'CinnBey_Beruwala_TA_Cleaned.xlsx'

CinnBey_Beruwala_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as CinnBey_Beruwala_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Hikka Tranz by Cinnamon

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 7. HikkaTranzbyCinnamon-TripAdvisor.xlsx to 7. HikkaTranzbyCinnamon-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '7. HikkaTranzbyCinnamon-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
Cinn_HikkaTranz_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'Cinn_HikkaTranz_TA'")
print("Shape (Rows, Columns):", Cinn_HikkaTranz_TA.shape)


TripAdvisor file loaded successfully as 'Cinn_HikkaTranz_TA'
Shape (Rows, Columns): (1278, 269)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(Cinn_HikkaTranz_TA.columns.tolist())

print("\nData types and info:")
Cinn_HikkaTranz_TA.info()


All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = Cinn_HikkaTranz_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(Cinn_HikkaTranz_TA) * 100).round(2))


Missing values per column:
ownerResponse                        1278
ownerResponse/connectionToSubject      14
ownerResponse/id                       14
ownerResponse/lang                     14
ownerResponse/publishedDate            14
                                     ... 
user/userLocation                    1278
user/userLocation/id                  714
user/userLocation/name                714
user/userLocation/shortName           714
user/username                           1
Length: 239, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            55.87
user/userLocation/name          55.87
user/userLocation/shortName     55.87
user/username                    0.08
Length: 269, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = Cinn_HikkaTranz_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(Cinn_HikkaTranz_TA[Cinn_HikkaTranz_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:

# Unique values per column
print("Unique values count per column:")
for col in Cinn_HikkaTranz_TA.columns:
    print(f"{col}: {Cinn_HikkaTranz_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 5 unique values
id: 1278 unique values
lang: 18 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 2 unique values
ownerResponse/id: 1264 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 285 unique values
ownerResponse/responder: 6 unique values
ownerResponse/text: 1108 unique values
photos/0/height: 92 unique values
photos/0/id: 431 unique values
photos/0/image: 431 unique values
photos/0/width: 100 unique values
photos/1/height: 70 unique values
photos/1/id: 312 unique values
photos/1/image: 312 unique values
photos/1/width: 74 unique values
photos/2/height: 59 unique values
photos/2/id: 256 unique values
photos/2/image: 256 unique values
photos/2/width: 65 unique values
photos/3/height: 46 unique values
photos/3/id: 192 unique values
photos/3/image: 192 unique values
photos/3/width: 54 unique values
photos/4/height: 44 unique values
photos/4/id: 150 un

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in Cinn_HikkaTranz_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = Cinn_HikkaTranz_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in Cinn_HikkaTranz_TA.columns if col.startswith('subratings/')]
Cinn_HikkaTranz_TA = pd.concat([Cinn_HikkaTranz_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
Cinn_HikkaTranz_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform



In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(Cinn_HikkaTranz_TA.columns.tolist())
print(f"\nTotal columns: {len(Cinn_HikkaTranz_TA.columns)} | Total rows: {len(CinnGrand_Colombo_TA)}")

# Dataset info
print("\nDataset Info:")
Cinn_HikkaTranz_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(Cinn_HikkaTranz_TA[sub_rating_cols].head(10))


Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
4,5.0,5.0,5.0,5.0,5.0,4.0,TripAdvisor
5,5.0,5.0,5.0,5.0,4.0,5.0,TripAdvisor
6,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
7,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',
    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',

    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',
    'photos/44/height',
    'photos/44/id',
    'photos/44/image',
    'photos/44/width',
    'photos/45/height',
    'photos/45/id',
    'photos/45/image',
    'photos/45/width',
    'photos/46/height',
    'photos/46/id',
    'photos/46/image',
    'photos/46/width',
    'photos/47/height',
    'photos/47/id',
    'photos/47/image',
    'photos/47/width',

    'photos/48/height',
    'photos/48/id',
    'photos/48/image',
    'photos/48/width',
    'photos/49/height',
    'photos/49/id',
    'photos/49/image',
    'photos/49/width',

    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
Cinn_HikkaTranz_TA = Cinn_HikkaTranz_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
Cinn_HikkaTranz_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278 entries, 0 to 1277
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   1278 non-null   object 
 1   ownerResponse/publishedDate            1264 non-null   object 
 2   ownerResponse/text                     1264 non-null   object 
 3   placeInfo/name                         1278 non-null   object 
 4   placeInfo/rating                       1278 non-null   float64
 5   publishedDate                          1278 non-null   object 
 6   publishedPlatform                      1278 non-null   object 
 7   rating                                 1278 non-null   int64  
 8   roomTip                                151 non-null    object 
 9   text                                   1278 non-null   object 
 10  title                                  1277 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
Cinn_HikkaTranz_TA = Cinn_HikkaTranz_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})

In [ ]:
Cinn_HikkaTranz_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278 entries, 0 to 1277
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    1278 non-null   object 
 1   OwnerResponsepublishedDate  1264 non-null   object 
 2   OwnerResponse               1264 non-null   object 
 3   Hotel/Resort                1278 non-null   object 
 4   placeInfo/rating            1278 non-null   float64
 5   RatingPublishedDate         1278 non-null   object 
 6   RatingPublishedPlatform     1278 non-null   object 
 7   HotelRating                 1278 non-null   int64  
 8   RoomtipText                 151 non-null    object 
 9   ReviewText                  1278 non-null   object 
 10  ReviewTitle                 1277 non-null   object 
 11  TripType                    1278 non-null   object 
 12  TotalReviewsbyUser          1277 non-null   float64
 13  UserName                    1277 

In [ ]:

display(Cinn_HikkaTranz_TA.head(10))


,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-06,MOBILE,5,NaN,We had a wonderful experience at Hikka Tranz l...,...,Demon sanze,"Las Vegas, Nevada",Las Vegas,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-06,MOBILE,5,NaN,We had a wonderful experience at Hikka Tranz b...,...,john calsen,"New York City, New York",New York City,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
2,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-06,OTHER,5,NaN,We had such an amazing stay at this hotel! Our...,...,Natascha H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-05,MOBILE,5,NaN,We really appreciated the hospitality we recei...,...,Margaret Tompsan,"Rome, Italy",Rome,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
4,de,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-05,OTHER,5,Mein Tipp für zukünftig Reisende wäre auf jede...,Ich habe meinen Aufenthalt in diesem Hotel seh...,...,max B,NaN,NaN,5.0,5.0,5.0,5.0,4.0,5.0,TripAdvisor
5,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-05,OTHER,5,NaN,Absolutely beautiful location\nSo much to see ...,...,dayoStaffordshire,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.0,TripAdvisor
6,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-04,OTHER,5,NaN,"Location super, atmosphere very cool. Service ...",...,VALERII S,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
7,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-04,OTHER,5,NaN,"A wonderful hotel! My husband, I, and I stayed...",...,Рожкова М,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,en,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-04,OTHER,5,NaN,I had an amazing stay at Hikka Tranz — truly a...,...,Robert S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,de,NaN,NaN,Hikka Tranz by Cinnamon,4.3,2025-12-04,MOBILE,4,NaN,"Top Strandlage, großzügiger Garten- und Poolbe...",...,Sonja,"Vienna, Austria",Vienna,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'Cinn_HikkaTranz_TA_Cleaned.xlsx'

Cinn_HikkaTranz_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as Cinn_HikkaTranz_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Trinco Blu by Cinnamon

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 8. TrincoBlubyCinnamon-TripAdvisor.xlsx to 8. TrincoBlubyCinnamon-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '8. TrincoBlubyCinnamon-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
Cinn_TrincoBlu_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'Cinn_TrincoBlu_TA'")
print("Shape (Rows, Columns):", Cinn_TrincoBlu_TA.shape)

TripAdvisor file loaded successfully as 'Cinn_TrincoBlu_TA'
Shape (Rows, Columns): (2499, 269)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(Cinn_TrincoBlu_TA.columns.tolist())

print("\nData types and info:")
Cinn_TrincoBlu_TA.info()


All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = Cinn_TrincoBlu_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(Cinn_TrincoBlu_TA) * 100).round(2))



Missing values per column:
ownerResponse                        2499
ownerResponse/connectionToSubject     191
ownerResponse/id                      191
ownerResponse/lang                    191
ownerResponse/publishedDate           191
                                     ... 
user/userLocation                    2499
user/userLocation/id                 1004
user/userLocation/name               1004
user/userLocation/shortName          1004
user/username                           5
Length: 241, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            40.18
user/userLocation/name          40.18
user/userLocation/shortName     40.18
user/username                    0.20
Length: 269, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = Cinn_TrincoBlu_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(Cinn_TrincoBlu_TA[Cinn_TrincoBlu_TA.duplicated()].head())


Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in Cinn_TrincoBlu_TA.columns:
    print(f"{col}: {Cinn_TrincoBlu_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 9 unique values
id: 2499 unique values
lang: 16 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 10 unique values
ownerResponse/id: 2308 unique values
ownerResponse/lang: 9 unique values
ownerResponse/publishedDate: 911 unique values
ownerResponse/responder: 18 unique values
ownerResponse/text: 1993 unique values
photos/0/height: 180 unique values
photos/0/id: 712 unique values
photos/0/image: 711 unique values
photos/0/width: 150 unique values
photos/1/height: 153 unique values
photos/1/id: 533 unique values
photos/1/image: 533 unique values
photos/1/width: 125 unique values
photos/2/height: 132 unique values
photos/2/id: 417 unique values
photos/2/image: 417 unique values
photos/2/width: 114 unique values
photos/3/height: 104 unique values
photos/3/id: 305 unique values
photos/3/image: 305 unique values
photos/3/width: 99 unique values
photos/4/height: 84 unique values
photos/4/id

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in Cinn_TrincoBlu_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = Cinn_TrincoBlu_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in Cinn_TrincoBlu_TA.columns if col.startswith('subratings/')]
Cinn_TrincoBlu_TA = pd.concat([Cinn_TrincoBlu_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
Cinn_TrincoBlu_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',
    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',

    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',
    'photos/44/height',
    'photos/44/id',
    'photos/44/image',
    'photos/44/width',
    'photos/45/height',
    'photos/45/id',
    'photos/45/image',
    'photos/45/width',
    'photos/46/height',
    'photos/46/id',
    'photos/46/image',
    'photos/46/width',
    'photos/47/height',
    'photos/47/id',
    'photos/47/image',
    'photos/47/width',

    'photos/48/height',
    'photos/48/id',
    'photos/48/image',
    'photos/48/width',
    'photos/49/height',
    'photos/49/id',
    'photos/49/image',
    'photos/49/width',

    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
Cinn_TrincoBlu_TA = Cinn_TrincoBlu_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
Cinn_TrincoBlu_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   2499 non-null   object 
 1   ownerResponse/publishedDate            2308 non-null   object 
 2   ownerResponse/text                     2308 non-null   object 
 3   placeInfo/name                         2499 non-null   object 
 4   placeInfo/rating                       2499 non-null   float64
 5   publishedDate                          2499 non-null   object 
 6   publishedPlatform                      2499 non-null   object 
 7   rating                                 2499 non-null   int64  
 8   roomTip                                513 non-null    object 
 9   text                                   2499 non-null   object 
 10  title                                  2497 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
Cinn_TrincoBlu_TA = Cinn_TrincoBlu_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})


In [ ]:
Cinn_TrincoBlu_TA.info()







<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    2499 non-null   object 
 1   OwnerResponsepublishedDate  2308 non-null   object 
 2   OwnerResponse               2308 non-null   object 
 3   Hotel/Resort                2499 non-null   object 
 4   placeInfo/rating            2499 non-null   float64
 5   RatingPublishedDate         2499 non-null   object 
 6   RatingPublishedPlatform     2499 non-null   object 
 7   HotelRating                 2499 non-null   int64  
 8   RoomtipText                 513 non-null    object 
 9   ReviewText                  2499 non-null   object 
 10  ReviewTitle                 2497 non-null   object 
 11  TripType                    2498 non-null   object 
 12  TotalReviewsbyUser          2494 non-null   float64
 13  UserName                    2494 

In [ ]:
display(Cinn_TrincoBlu_TA.head(10))





,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Trinco Blu By Cinnamon,4.3,2025-12-30,OTHER,4,NaN,We stayed at Trinco Blu by Cinnamon with our 1...,...,Amayuru K,NaN,NaN,5.0,5.0,5.0,5.0,5.0,3.0,TripAdvisor
1,en,2025-12-29,"Dear Mr. Paveera, Ayubowan ! Thank you for you...",Trinco Blu By Cinnamon,4.3,2025-12-28,OTHER,4,"If possible, try out their beach chalets, grea...",Spent a night here. Beach Challets were terrif...,...,Paveera D,NaN,NaN,5.0,4.0,5.0,5.0,5.0,5.0,TripAdvisor
2,en,2025-12-29,"Dear Ms. Nuzrath, Ayubowan ! Thank you for you...",Trinco Blu By Cinnamon,4.3,2025-12-28,OTHER,5,NaN,We took this trip for relaxation after a busy ...,...,Nuzrath S,"Colombo, Sri Lanka",Colombo,5.0,5.0,5.0,5.0,5.0,4.0,TripAdvisor
3,en,2025-12-29,"Dear Mr. Lakshithaa, Ayubowan ! Thank you for ...",Trinco Blu By Cinnamon,4.3,2025-12-27,OTHER,5,NaN,A big shout to Resident Manager & her amazing ...,...,lakshitha A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
4,en,2025-12-29,"Dear Mr. Chamira, Ayubowan ! Thank you for you...",Trinco Blu By Cinnamon,4.3,2025-12-26,OTHER,5,NaN,It was nice experience near the beach front. T...,...,Chamira B,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.0,TripAdvisor
5,en,2025-12-25,"Dear Mr. Dakshinis, Ayubowan ! Thank you for y...",Trinco Blu By Cinnamon,4.3,2025-12-24,OTHER,5,NaN,We truly enjoyed our stay and had a wonderful ...,...,Dakshini S,NaN,NaN,4.0,5.0,5.0,5.0,4.0,5.0,TripAdvisor
6,en,2025-12-25,"Dear Mr.Nidarshana, Ayubowan ! Thank you for y...",Trinco Blu By Cinnamon,4.3,2025-12-22,OTHER,3,NaN,"Overall, our stay was good. The room service a...",...,nidarshana1998,"Colombo, Sri Lanka",Colombo,4.0,5.0,5.0,2.0,5.0,3.0,TripAdvisor
7,en,2025-12-22,"Dear Ms. Amalie, Ayubowan ! Thank you for your...",Trinco Blu By Cinnamon,4.3,2025-12-21,OTHER,5,NaN,The best customer service ever!\nHuge thanks t...,...,Amalie S,NaN,NaN,5.0,5.0,4.0,5.0,5.0,5.0,TripAdvisor
8,en,2025-12-22,"Dear Mr. Sanj, Ayubowan ! Thank you for your w...",Trinco Blu By Cinnamon,4.3,2025-12-21,OTHER,5,Best time to do excursions is march to aug,I had an excellent stay at Trinco Blu by Cinna...,...,sanj s,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,en,2025-12-21,"Dear Ms. Dilani, Ayubowan ! Thank you for your...",Trinco Blu By Cinnamon,4.3,2025-12-20,OTHER,5,NaN,We had a wonderful experience at Trinco Blu. A...,...,Dilani S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:



# Save the cleaned DataFrame as Excel
excel_filename = 'Cinn_TrincoBlu_TA_Cleaned.xlsx'

Cinn_TrincoBlu_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)



Dataset saved as Cinn_TrincoBlu_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cinnamon Lodge Habarana

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 9. CinnamonLodgeHabarana-TripAdvisor.xlsx to 9. CinnamonLodgeHabarana-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '9. CinnamonLodgeHabarana-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
CinnLodge_Habarana_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'CinnLodge_Habarana_TA'")
print("Shape (Rows, Columns):", CinnLodge_Habarana_TA.shape)


TripAdvisor file loaded successfully as 'CinnLodge_Habarana_TA'
Shape (Rows, Columns): (1760, 193)


In [ ]:

# Printing a list of all the columns in the dataset
print("All columns:")
print(CinnLodge_Habarana_TA.columns.tolist())

print("\nData types and info:")
CinnLodge_Habarana_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = CinnLodge_Habarana_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(CinnLodge_Habarana_TA) * 100).round(2))

Missing values per column:
ownerResponse                        1760
ownerResponse/connectionToSubject      51
ownerResponse/id                       51
ownerResponse/lang                     51
ownerResponse/publishedDate            51
                                     ... 
user/userLocation                    1760
user/userLocation/id                  989
user/userLocation/name                989
user/userLocation/shortName           989
user/username                           1
Length: 164, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            56.19
user/userLocation/name          56.19
user/userLocation/shortName     56.19
user/username                    0.06
Length: 193, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = CinnLodge_Habarana_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(CinnLodge_Habarana_TA[CinnLodge_Habarana_TA.duplicated()].head())


Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in CinnLodge_Habarana_TA.columns:
    print(f"{col}: {CinnLodge_Habarana_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 4 unique values
id: 1760 unique values
lang: 13 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 1 unique values
ownerResponse/id: 1709 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 608 unique values
ownerResponse/responder: 14 unique values
ownerResponse/text: 1436 unique values
photos/0/height: 112 unique values
photos/0/id: 505 unique values
photos/0/image: 505 unique values
photos/0/width: 98 unique values
photos/1/height: 83 unique values
photos/1/id: 378 unique values
photos/1/image: 378 unique values
photos/1/width: 82 unique values
photos/2/height: 65 unique values
photos/2/id: 303 unique values
photos/2/image: 303 unique values
photos/2/width: 69 unique values
photos/3/height: 58 unique values
photos/3/id: 222 unique values
photos/3/image: 222 unique values
photos/3/width: 62 unique values
photos/4/height: 50 unique values
photos/4/id: 166 u

In [ ]:

import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in CinnLodge_Habarana_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = CinnLodge_Habarana_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in CinnLodge_Habarana_TA.columns if col.startswith('subratings/')]
CinnLodge_Habarana_TA = pd.concat([CinnLodge_Habarana_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
CinnLodge_Habarana_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform

In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(CinnLodge_Habarana_TA.columns.tolist())
print(f"\nTotal columns: {len(CinnLodge_Habarana_TA.columns)} | Total rows: {len(CinnLodge_Habarana_TA)}")

# Dataset info
print("\nDataset Info:")
CinnLodge_Habarana_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(CinnLodge_Habarana_TA[sub_rating_cols].head(10))

Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,4.0,5.0,4.0,5.0,5.0,5.0,TripAdvisor
2,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,5.0,5.0,4.0,NaN,4.0,4.0,TripAdvisor
4,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
5,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,5.0,4.0,5.0,5.0,4.0,5.0,TripAdvisor
7,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',
    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',




    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
CinnLodge_Habarana_TA = CinnLodge_Habarana_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
CinnLodge_Habarana_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   1760 non-null   object 
 1   ownerResponse/publishedDate            1709 non-null   object 
 2   ownerResponse/text                     1709 non-null   object 
 3   placeInfo/name                         1760 non-null   object 
 4   placeInfo/rating                       1760 non-null   float64
 5   publishedDate                          1760 non-null   object 
 6   publishedPlatform                      1760 non-null   object 
 7   rating                                 1760 non-null   int64  
 8   roomTip                                271 non-null    object 
 9   text                                   1760 non-null   object 
 10  title                                  1755 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
CinnLodge_Habarana_TA = CinnLodge_Habarana_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})


In [ ]:
CinnLodge_Habarana_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    1760 non-null   object 
 1   OwnerResponsepublishedDate  1709 non-null   object 
 2   OwnerResponse               1709 non-null   object 
 3   Hotel/Resort                1760 non-null   object 
 4   placeInfo/rating            1760 non-null   float64
 5   RatingPublishedDate         1760 non-null   object 
 6   RatingPublishedPlatform     1760 non-null   object 
 7   HotelRating                 1760 non-null   int64  
 8   RoomtipText                 271 non-null    object 
 9   ReviewText                  1760 non-null   object 
 10  ReviewTitle                 1755 non-null   object 
 11  TripType                    1759 non-null   object 
 12  TotalReviewsbyUser          1759 non-null   float64
 13  UserName                    1759 

In [ ]:
display(CinnLodge_Habarana_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,2025-12-06,"Dear Ryan,\n\nGreetings from Cinnamon Lodge Ha...",Cinnamon Lodge Habarana,4.6,2025-12-06,MOBILE,5,NaN,This hotel was a highlight of my travels throu...,...,Ryan C,"Birmingham, United Kingdom",Birmingham,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
1,en,2024-10-04,"Dear Guest,\nGreetings From Cinnamon Lodge Hab...",Cinnamon Lodge Habarana,4.6,2024-09-27,OTHER,5,"This is an excellent spot, but maybe not so co...",We just came back from Srilanka yesterday. Thi...,...,Hemant J,"Pune, India",Pune,4.0,5.0,4.0,5.0,5.0,5.0,TripAdvisor
2,en,2025-12-06,"Dear Guest,\n\nGreetings from Cinnamon Lodge H...",Cinnamon Lodge Habarana,4.6,2025-12-05,OTHER,4,NaN,This is the first time we stayed at a Cinnamon...,...,hotelreviews06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
3,en,2025-12-06,"Dear Guest,\n\nGreetings from Cinnamon Lodge H...",Cinnamon Lodge Habarana,4.6,2025-12-04,OTHER,5,NaN,We loved our overnight stay here. Even as you ...,...,centralvic,"Ballarat, Australia",Ballarat,5.0,5.0,4.0,NaN,4.0,4.0,TripAdvisor
4,en,2025-12-06,"Dear Paul,\n\nGreetings from Cinnamon Lodge Ha...",Cinnamon Lodge Habarana,4.6,2025-12-04,MOBILE,5,NaN,Only an overnight stay after climbing Sigiriya...,...,paul c,"Stamford, United Kingdom",Stamford,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
5,en,2025-12-06,"Dear Guest,\n\nGreetings from Cinnamon Lodge H...",Cinnamon Lodge Habarana,4.6,2025-12-02,OTHER,5,NaN,Sandeep was so friendly and made sure me and m...,...,Hgggg B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,en,2025-12-06,"Dear Guest,\n\nGreetings from Cinnamon Lodge H...",Cinnamon Lodge Habarana,4.6,2025-12-01,OTHER,4,NaN,"Lovely grounds, large pleasant room, really ni...",...,Geoff H,NaN,NaN,5.0,4.0,5.0,5.0,5.0,4.0,TripAdvisor
7,en,2025-12-06,"Dear Dasun,\n\nGreetings from Cinnamon Lodge H...",Cinnamon Lodge Habarana,4.6,2025-12-01,OTHER,5,NaN,The rooms ware very comfortable and spacious. ...,...,Dasun W,Sri Lanka,Sri Lanka,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
8,de,2025-12-06,"Dear Guest,\n\nGreetings from Cinnamon Lodge H...",Cinnamon Lodge Habarana,4.6,2025-11-30,MOBILE,5,Italienischen Kaffee gibt es in Cafés an der S...,Schöne großzügige Bungalows in einem großen Pa...,...,Kran,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
9,de,2025-12-06,"Dear Guest,\n\nGreetings from Cinnamon Lodge H...",Cinnamon Lodge Habarana,4.6,2025-11-29,MOBILE,5,Es wird immer geholfen! \nWir waren zur Übersc...,Top Location \nService und Essen grandios;\nBe...,...,Irmi F,"Hohenwart, Germany",Hohenwart,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:

# Save the cleaned DataFrame as Excel
excel_filename = 'CinnLodge_Habarana_TA_Cleaned.xlsx'

CinnLodge_Habarana_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as CinnLodge_Habarana_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Habarana Village by Cinnamon

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()

Saving 10. HabaranaVillagebyCinnamon-TripAdvisor.xlsx to 10. HabaranaVillagebyCinnamon-TripAdvisor.xlsx


In [ ]:

# Exact file name after upload (change if different)
trip_file_path = '10. HabaranaVillagebyCinnamon-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
Cinn_HabaranaVillage_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'Cinn_HabaranaVillage_TA'")
print("Shape (Rows, Columns):", Cinn_HabaranaVillage_TA.shape)

TripAdvisor file loaded successfully as 'Cinn_HabaranaVillage_TA'
Shape (Rows, Columns): (2499, 265)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(Cinn_HabaranaVillage_TA.columns.tolist())

print("\nData types and info:")
Cinn_HabaranaVillage_TA.info()


All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = Cinn_HabaranaVillage_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(Cinn_HabaranaVillage_TA) * 100).round(2))

Missing values per column:
ownerResponse                        2499
ownerResponse/connectionToSubject     244
ownerResponse/id                      244
ownerResponse/lang                    244
ownerResponse/publishedDate           244
                                     ... 
user/userLocation                    2499
user/userLocation/id                  933
user/userLocation/name                933
user/userLocation/shortName           933
user/username                           2
Length: 236, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            37.33
user/userLocation/name          37.33
user/userLocation/shortName     37.33
user/username                    0.08
Length: 265, dtype: float64


In [ ]:

# Check duplicate rows
duplicates_ta = Cinn_HabaranaVillage_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(Cinn_HabaranaVillage_TA[Cinn_HabaranaVillage_TA.duplicated()].head())


Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in Cinn_HabaranaVillage_TA.columns:
    print(f"{col}: {Cinn_HabaranaVillage_TA[col].nunique()} unique values")



Unique values count per column:
helpfulVotes: 8 unique values
id: 2499 unique values
lang: 17 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 18 unique values
ownerResponse/id: 2255 unique values
ownerResponse/lang: 8 unique values
ownerResponse/publishedDate: 775 unique values
ownerResponse/responder: 23 unique values
ownerResponse/text: 1485 unique values
photos/0/height: 160 unique values
photos/0/id: 635 unique values
photos/0/image: 635 unique values
photos/0/width: 145 unique values
photos/1/height: 135 unique values
photos/1/id: 484 unique values
photos/1/image: 484 unique values
photos/1/width: 124 unique values
photos/2/height: 106 unique values
photos/2/id: 371 unique values
photos/2/image: 371 unique values
photos/2/width: 101 unique values
photos/3/height: 94 unique values
photos/3/id: 268 unique values
photos/3/image: 268 unique values
photos/3/width: 90 unique values
photos/4/height: 74 unique values
photos/4/id:

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in Cinn_HabaranaVillage_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = Cinn_HabaranaVillage_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in Cinn_HabaranaVillage_TA.columns if col.startswith('subratings/')]
Cinn_HabaranaVillage_TA = pd.concat([Cinn_HabaranaVillage_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
Cinn_HabaranaVillage_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform




In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(Cinn_HabaranaVillage_TA.columns.tolist())
print(f"\nTotal columns: {len(Cinn_HabaranaVillage_TA.columns)} | Total rows: {len(Cinn_HabaranaVillage_TA)}")

# Dataset info
print("\nDataset Info:")
Cinn_HabaranaVillage_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(Cinn_HabaranaVillage_TA[sub_rating_cols].head(10))



Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,5.0,4.0,5.0,5.0,3.0,5.0,TripAdvisor
1,5.0,4.0,4.0,5.0,5.0,5.0,TripAdvisor
2,5.0,5.0,5.0,5.0,5.0,4.0,TripAdvisor
3,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
7,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',
    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',

    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',
    'photos/44/height',
    'photos/44/id',
    'photos/44/image',
    'photos/44/width',
    'photos/45/height',
    'photos/45/id',
    'photos/45/image',
    'photos/45/width',
    'photos/46/height',
    'photos/46/id',
    'photos/46/image',
    'photos/46/width',
    'photos/47/height',
    'photos/47/id',
    'photos/47/image',
    'photos/47/width',

    'photos/48/height',
    'photos/48/id',
    'photos/48/image',
    'photos/48/width',


    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
Cinn_HabaranaVillage_TA = Cinn_HabaranaVillage_TA.drop(columns=columns_to_drop, errors='ignore')


In [ ]:
Cinn_HabaranaVillage_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   2499 non-null   object 
 1   ownerResponse/publishedDate            2255 non-null   object 
 2   ownerResponse/text                     2255 non-null   object 
 3   placeInfo/name                         2499 non-null   object 
 4   placeInfo/rating                       2499 non-null   float64
 5   publishedDate                          2499 non-null   object 
 6   publishedPlatform                      2499 non-null   object 
 7   rating                                 2499 non-null   int64  
 8   roomTip                                364 non-null    object 
 9   text                                   2499 non-null   object 
 10  title                                  2496 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
Cinn_HabaranaVillage_TA = Cinn_HabaranaVillage_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})



In [ ]:
Cinn_HabaranaVillage_TA.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    2499 non-null   object 
 1   OwnerResponsepublishedDate  2255 non-null   object 
 2   OwnerResponse               2255 non-null   object 
 3   Hotel/Resort                2499 non-null   object 
 4   placeInfo/rating            2499 non-null   float64
 5   RatingPublishedDate         2499 non-null   object 
 6   RatingPublishedPlatform     2499 non-null   object 
 7   HotelRating                 2499 non-null   int64  
 8   RoomtipText                 364 non-null    object 
 9   ReviewText                  2499 non-null   object 
 10  ReviewTitle                 2496 non-null   object 
 11  TripType                    2499 non-null   object 
 12  TotalReviewsbyUser          2497 non-null   float64
 13  UserName                    2497 

In [ ]:
display(Cinn_HabaranaVillage_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,2025-12-26,Dear Sir / Madam.\n\nGreetings from Habarana V...,Habarana Village By Cinnamon,4.5,2025-12-25,OTHER,4,NaN,A Picturesque Family Escape with Room for Culi...,...,Kushan Amarasinghe,"Colombo, Sri Lanka",Colombo,5.0,4.0,5.0,5.0,5.0,3.0,TripAdvisor
1,en,2024-09-12,"Dear Guest,\n\nWe're delighted to hear that yo...",Habarana Village By Cinnamon,4.5,2024-09-11,OTHER,5,NaN,We stayed for three days and found the staff s...,...,Sue,"Buckinghamshire, United Kingdom",Buckinghamshire,5.0,4.0,4.0,5.0,5.0,5.0,TripAdvisor
2,en,2025-12-25,Dear Sir / Madam.\n\nGreetings from Habarana V...,Habarana Village By Cinnamon,4.5,2025-12-24,OTHER,5,NaN,"it was a great experience , very friendly sta...",...,cnayakarathne,"Kurunegala, Sri Lanka",Kurunegala,5.0,5.0,5.0,5.0,4.0,5.0,TripAdvisor
3,en,2025-12-25,Dear Sir / Madam.\n\nGreetings from Habarana V...,Habarana Village By Cinnamon,4.5,2025-12-21,OTHER,5,NaN,Our family of 6 had a wonderful stay at Habara...,...,Charles H,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,en,2025-12-18,Dear Sir / Madam.\n\nGreetings from Habarana ...,Habarana Village By Cinnamon,4.5,2025-12-17,OTHER,5,NaN,"As always, Cinnamon hospitality was great duri...",...,Dammika,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,en,2025-12-18,Dear Sir / Madam.\n\nGreetings from Habarana V...,Habarana Village By Cinnamon,4.5,2025-12-17,MOBILE,5,NaN,"The resort is worderful, with a majestic view ...",...,Manolo Bedin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,en,2025-12-18,Dear Mr. Shamil.\n\nGreetings from Habarana Vi...,Habarana Village By Cinnamon,4.5,2025-12-17,MOBILE,5,NaN,We are currently having a wonderful stay at Ci...,...,Shamil J,"Colombo, Sri Lanka",Colombo,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
7,en,2025-12-12,Dear Mr. Janitha.\n\nGreetings from Habarana V...,Habarana Village By Cinnamon,4.5,2025-12-12,MOBILE,5,NaN,We had a fantastic two-night family stay at Ha...,...,Janitha W,"Colombo, Sri Lanka",Colombo,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,en,2025-12-12,Dear Sir /Madam.\n\nGreetings from Habarana Vi...,Habarana Village By Cinnamon,4.5,2025-12-11,OTHER,5,NaN,We stayed at Habarana Village on the 6th of De...,...,Loshitha P,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
9,en,2025-12-11,Dear Sir / Madam.\n\nGreetings from Habarana V...,Habarana Village By Cinnamon,4.5,2025-12-10,OTHER,5,NaN,Very good location and there’s a lake nearby. ...,...,Yumeth S,"Colombo, Sri Lanka",Colombo,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'Cinn_HabaranaVillage_TA_Cleaned.xlsx'

Cinn_HabaranaVillage_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)

Dataset saved as Cinn_HabaranaVillage_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cinnamon Citadel Kandy

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()  # Upload file named e.g., "1. CinnamonGrandColombo-TripAdvisor.xlsx"

Saving 11. CinnamonCitadelKandy-TripAdvisor.xlsx to 11. CinnamonCitadelKandy-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '11. CinnamonCitadelKandy-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
Cinn_CitadelKandy_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'Cinn_CitadelKandy_TA'")
print("Shape (Rows, Columns):", Cinn_CitadelKandy_TA.shape)

TripAdvisor file loaded successfully as 'Cinn_CitadelKandy_TA'
Shape (Rows, Columns): (2500, 253)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(Cinn_CitadelKandy_TA.columns.tolist())

print("\nData types and info:")
Cinn_CitadelKandy_TA.info()


All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = Cinn_CitadelKandy_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(Cinn_CitadelKandy_TA) * 100).round(2))

Missing values per column:
ownerResponse                        2500
ownerResponse/connectionToSubject       7
ownerResponse/id                        7
ownerResponse/lang                      7
ownerResponse/publishedDate             7
                                     ... 
user/userLocation                    2500
user/userLocation/id                 1008
user/userLocation/name               1008
user/userLocation/shortName          1008
user/username                           2
Length: 223, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            40.32
user/userLocation/name          40.32
user/userLocation/shortName     40.32
user/username                    0.08
Length: 253, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = Cinn_CitadelKandy_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(Cinn_CitadelKandy_TA[Cinn_CitadelKandy_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in Cinn_CitadelKandy_TA.columns:
    print(f"{col}: {Cinn_CitadelKandy_TA[col].nunique()} unique values")


Unique values count per column:
helpfulVotes: 6 unique values
id: 2500 unique values
lang: 18 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 11 unique values
ownerResponse/id: 2493 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 1227 unique values
ownerResponse/responder: 8 unique values
ownerResponse/text: 2090 unique values
photos/0/height: 167 unique values
photos/0/id: 633 unique values
photos/0/image: 633 unique values
photos/0/width: 156 unique values
photos/1/height: 127 unique values
photos/1/id: 442 unique values
photos/1/image: 442 unique values
photos/1/width: 125 unique values
photos/2/height: 104 unique values
photos/2/id: 318 unique values
photos/2/image: 318 unique values
photos/2/width: 100 unique values
photos/3/height: 84 unique values
photos/3/id: 220 unique values
photos/3/image: 220 unique values
photos/3/width: 79 unique values
photos/4/height: 63 unique values
photos/4/id:

In [ ]:

import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in Cinn_CitadelKandy_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = Cinn_CitadelKandy_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in Cinn_CitadelKandy_TA.columns if col.startswith('subratings/')]
Cinn_CitadelKandy_TA = pd.concat([Cinn_CitadelKandy_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
Cinn_CitadelKandy_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform



In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(Cinn_CitadelKandy_TA.columns.tolist())
print(f"\nTotal columns: {len(Cinn_CitadelKandy_TA.columns)} | Total rows: {len(Cinn_CitadelKandy_TA)}")

# Dataset info
print("\nDataset Info:")
Cinn_CitadelKandy_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(Cinn_CitadelKandy_TA[sub_rating_cols].head(10))

Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,5.0,3.0,5.0,5.0,5.0,5.0,TripAdvisor
1,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
2,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
4,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
6,5.0,5.0,NaN,NaN,5.0,NaN,TripAdvisor
7,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,5.0,4.0,5.0,5.0,5.0,5.0,TripAdvisor
9,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',
    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',

    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',
    'photos/44/height',
    'photos/44/id',
    'photos/44/image',
    'photos/44/width',
    'photos/45/height',
    'photos/45/id',
    'photos/45/image',
    'photos/45/width',

    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
Cinn_CitadelKandy_TA = Cinn_CitadelKandy_TA.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
Cinn_CitadelKandy_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   2500 non-null   object 
 1   ownerResponse/publishedDate            2493 non-null   object 
 2   ownerResponse/text                     2493 non-null   object 
 3   placeInfo/name                         2500 non-null   object 
 4   placeInfo/rating                       2500 non-null   float64
 5   publishedDate                          2500 non-null   object 
 6   publishedPlatform                      2500 non-null   object 
 7   rating                                 2500 non-null   int64  
 8   roomTip                                354 non-null    object 
 9   text                                   2500 non-null   object 
 10  title                                  2490 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
Cinn_CitadelKandy_TA = Cinn_CitadelKandy_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})

In [ ]:
Cinn_CitadelKandy_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    2500 non-null   object 
 1   OwnerResponsepublishedDate  2493 non-null   object 
 2   OwnerResponse               2493 non-null   object 
 3   Hotel/Resort                2500 non-null   object 
 4   placeInfo/rating            2500 non-null   float64
 5   RatingPublishedDate         2500 non-null   object 
 6   RatingPublishedPlatform     2500 non-null   object 
 7   HotelRating                 2500 non-null   int64  
 8   RoomtipText                 354 non-null    object 
 9   ReviewText                  2500 non-null   object 
 10  ReviewTitle                 2490 non-null   object 
 11  TripType                    2500 non-null   object 
 12  TotalReviewsbyUser          2499 non-null   float64
 13  UserName                    2499 

In [ ]:
display(Cinn_CitadelKandy_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,2025-12-10,"Dear Guest,\nGreetings from Cinnamon Citadel!\...",Cinnamon Citadel Kandy,4.3,2025-12-10,OTHER,5,Maybe the hotel could have a supermarket or sh...,A beautiful hotel just outside Kandy. Amazing ...,...,Charlieepaigeeee1993,"Evesham, United Kingdom",Evesham,5.0,3.0,5.0,5.0,5.0,5.0,TripAdvisor
1,en,2025-12-10,"Dear Achintha,\nGreetings from Cinnamon Citade...",Cinnamon Citadel Kandy,4.3,2025-12-09,OTHER,5,NaN,I booked 1 Night Night stay in this hotel for ...,...,Achintha,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
2,en,2025-12-08,"Dear Guest,\nGreetings from Cinnamon Citadel!\...",Cinnamon Citadel Kandy,4.3,2025-12-07,OTHER,5,NaN,A beautiful hotel and location on edge of Kand...,...,Diane R,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,en,2025-12-07,"Dear Guest,\nThank you so much for sharing you...",Cinnamon Citadel Kandy,4.3,2025-12-07,MOBILE,5,NaN,We arrived as the cyclone weather took hold of...,...,0rthopaedics,"Sligo, Ireland",Sligo,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
4,en,2025-12-07,"Dear Monali,\nThank you for taking the time to...",Cinnamon Citadel Kandy,4.3,2025-12-07,OTHER,5,NaN,"Staff are wonderful, helpful and kind. Sasi a...",...,Monali H,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,en,2025-12-04,"Dear Emily,\nThank you for reaching out to us....",Cinnamon Citadel Kandy,4.3,2025-12-04,OTHER,5,NaN,Cinnamon Citadel Kandy is open and operating a...,...,emily f,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
6,en,2025-12-04,"Dear Guest,\nGreetings from Cinnamon Citadel!\...",Cinnamon Citadel Kandy,4.3,2025-12-04,OTHER,5,NaN,As always it is a pleasure to return to the Ci...,...,Kanch P,"Kandy, Sri Lanka",Kandy,5.0,5.0,NaN,NaN,NaN,5.0,TripAdvisor
7,en,2025-12-03,"Dear Guest,\nGreetings from Cinnamon Citadel!\...",Cinnamon Citadel Kandy,4.3,2025-12-03,OTHER,5,NaN,"Great hotel, great food, struggled to connect ...",...,pdschabik,"Isle of Man, United Kingdom",Isle of Man,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,en,2025-11-30,"Dear Guest,\nGreetings from Cinnamon Citadel!\...",Cinnamon Citadel Kandy,4.3,2025-11-27,OTHER,5,NaN,"Lovely rooms, overlooking the rising river! W...",...,Deidre F,NaN,NaN,5.0,4.0,5.0,5.0,5.0,5.0,TripAdvisor
9,en,2025-11-25,"Dear Melissa,\nThank you for sharing your expe...",Cinnamon Citadel Kandy,4.3,2025-11-25,MOBILE,4,This hotel is a good bit out of Kandy city cen...,"Staff where friendly and welcoming on arrival,...",...,Melissa R,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'Cinn_CitadelKandy_TA_Cleaned.xlsx'

Cinn_CitadelKandy_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)


Dataset saved as Cinn_CitadelKandy_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cinnamon Wild Yala

In [ ]:
# Upload the TripAdvisor Excel file for Cinnamon Grand Colombo
from google.colab import files
uploaded = files.upload()  # Upload file named e.g., "1. CinnamonGrandColombo-TripAdvisor.xlsx"

Saving 12. CinnamonWildYala-TripAdvisor.xlsx to 12. CinnamonWildYala-TripAdvisor.xlsx


In [ ]:
# Exact file name after upload (change if different)
trip_file_path = '12. CinnamonWildYala-TripAdvisor.xlsx'  # Update if needed

# Load Excel file
CinnWild_Yala_TA = pd.read_excel(trip_file_path)

print("TripAdvisor file loaded successfully as 'CinnWild_Yala_TA'")
print("Shape (Rows, Columns):", CinnWild_Yala_TA.shape)


TripAdvisor file loaded successfully as 'CinnWild_Yala_TA'
Shape (Rows, Columns): (2499, 309)


In [ ]:
# Printing a list of all the columns in the dataset
print("All columns:")
print(CinnWild_Yala_TA.columns.tolist())

print("\nData types and info:")
CinnWild_Yala_TA.info()

All columns:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id', 'photos/10/image', 'photos/10/

In [ ]:
# Check missing values
missing_ta = CinnWild_Yala_TA.isnull().sum()
print("Missing values per column:")
print(missing_ta[missing_ta > 0])

print("\nMissing percentage:")
print((missing_ta / len(CinnWild_Yala_TA) * 100).round(2))

Missing values per column:
ownerResponse                        2499
ownerResponse/connectionToSubject     253
ownerResponse/id                      253
ownerResponse/lang                    253
ownerResponse/publishedDate           253
                                     ... 
user/userLocation                    2499
user/userLocation/id                 1342
user/userLocation/name               1342
user/userLocation/shortName          1342
user/username                           2
Length: 280, dtype: int64

Missing percentage:
helpfulVotes                     0.00
id                               0.00
lang                             0.00
locationId                       0.00
ownerResponse                  100.00
                                ...  
user/userLocation              100.00
user/userLocation/id            53.70
user/userLocation/name          53.70
user/userLocation/shortName     53.70
user/username                    0.08
Length: 309, dtype: float64


In [ ]:
# Check duplicate rows
duplicates_ta = CinnWild_Yala_TA.duplicated().sum()
print(f"Total fully duplicate rows: {duplicates_ta}")

if duplicates_ta > 0:
    print("Example duplicates:")
    display(CinnWild_Yala_TA[CinnWild_Yala_TA.duplicated()].head())

Total fully duplicate rows: 0


In [ ]:
# Unique values per column
print("Unique values count per column:")
for col in CinnWild_Yala_TA.columns:
    print(f"{col}: {CinnWild_Yala_TA[col].nunique()} unique values")

Unique values count per column:
helpfulVotes: 6 unique values
id: 2499 unique values
lang: 18 unique values
locationId: 1 unique values
ownerResponse: 0 unique values
ownerResponse/connectionToSubject: 3 unique values
ownerResponse/id: 2246 unique values
ownerResponse/lang: 1 unique values
ownerResponse/publishedDate: 554 unique values
ownerResponse/responder: 6 unique values
ownerResponse/text: 2163 unique values
photos/0/height: 179 unique values
photos/0/id: 739 unique values
photos/0/image: 738 unique values
photos/0/width: 153 unique values
photos/1/height: 138 unique values
photos/1/id: 591 unique values
photos/1/image: 591 unique values
photos/1/width: 122 unique values
photos/2/height: 122 unique values
photos/2/id: 494 unique values
photos/2/image: 494 unique values
photos/2/width: 116 unique values
photos/3/height: 100 unique values
photos/3/id: 371 unique values
photos/3/image: 371 unique values
photos/3/width: 94 unique values
photos/4/height: 87 unique values
photos/4/id: 

In [ ]:
import pandas as pd

# Extract sub-ratings (keep all separate, no averaging)
def extract_subratings(row):
    sub_dict = {}
    for i in range(7):
        name_col = f'subratings/{i}/name'
        value_col = f'subratings/{i}/value'
        if name_col in CinnWild_Yala_TA.columns and pd.notna(row.get(name_col)):
            cat = row[name_col].strip()
            val = row.get(value_col)
            if pd.notna(val):
                sub_dict[cat] = val
    return pd.Series(sub_dict)

sub_df = CinnWild_Yala_TA.apply(extract_subratings, axis=1)

# Rename only the 6 sub-rating categories
mapping = {
    'Value': 'ValueforMoneyRating',
    'Location': 'LocationRating',
    'Cleanliness': 'CleanlinessRating',
    'Service': 'StaffRating',
    'Rooms': 'ComfortRating',
    'Sleep Quality': 'SleepQualityRating'
}
sub_df = sub_df.rename(columns=mapping)

# Drop only the original subratings/* columns
sub_cols = [col for col in CinnWild_Yala_TA.columns if col.startswith('subratings/')]
CinnWild_Yala_TA = pd.concat([CinnWild_Yala_TA.drop(sub_cols, axis=1), sub_df], axis=1)

# Add platform column to the same DataFrame
CinnWild_Yala_TA['platform'] = 'TripAdvisor'

# Silent – CinnGrand_Colombo_TA is now updated with clean sub-ratings and platform

In [ ]:
# Show updated columns
print("Updated columns after sub-rating extraction:")
print(CinnWild_Yala_TA.columns.tolist())
print(f"\nTotal columns: {len(CinnWild_Yala_TA.columns)} | Total rows: {len(CinnWild_Yala_TA)}")

# Dataset info
print("\nDataset Info:")
CinnWild_Yala_TA.info()

# Preview new sub-rating columns
sub_rating_cols = ['CleanlinessRating', 'LocationRating', 'ComfortRating',
                   'StaffRating', 'ValueforMoneyRating', 'SleepQualityRating', 'platform']
print("\nFirst 10 rows of new sub-ratings + platform:")
display(CinnWild_Yala_TA[sub_rating_cols].head(10))


Updated columns after sub-rating extraction:
['helpfulVotes', 'id', 'lang', 'locationId', 'ownerResponse', 'ownerResponse/connectionToSubject', 'ownerResponse/id', 'ownerResponse/lang', 'ownerResponse/publishedDate', 'ownerResponse/responder', 'ownerResponse/text', 'photos/0/height', 'photos/0/id', 'photos/0/image', 'photos/0/width', 'photos/1/height', 'photos/1/id', 'photos/1/image', 'photos/1/width', 'photos/2/height', 'photos/2/id', 'photos/2/image', 'photos/2/width', 'photos/3/height', 'photos/3/id', 'photos/3/image', 'photos/3/width', 'photos/4/height', 'photos/4/id', 'photos/4/image', 'photos/4/width', 'photos/5/height', 'photos/5/id', 'photos/5/image', 'photos/5/width', 'photos/6/height', 'photos/6/id', 'photos/6/image', 'photos/6/width', 'photos/7/height', 'photos/7/id', 'photos/7/image', 'photos/7/width', 'photos/8/height', 'photos/8/id', 'photos/8/image', 'photos/8/width', 'photos/9/height', 'photos/9/id', 'photos/9/image', 'photos/9/width', 'photos/10/height', 'photos/10/id'

,CleanlinessRating,LocationRating,ComfortRating,StaffRating,ValueforMoneyRating,SleepQualityRating,platform
0,5.0,5.0,5.0,5.0,4.0,5.0,TripAdvisor
1,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
2,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,5.0,5.0,5.0,5.0,4.0,5.0,TripAdvisor
7,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,5.0,5.0,4.0,5.0,4.0,4.0,TripAdvisor
9,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Dropping the columns that are not needed for the analysis
columns_to_drop = [
    'helpfulVotes',
    'id',
    'locationId',
    'ownerResponse',
    'ownerResponse/connectionToSubject',
    'ownerResponse/id',
    'ownerResponse/lang',
    'ownerResponse/responder',
    'photos/0/height',
    'photos/0/id',
    'photos/0/image',
    'photos/0/width',
    'photos/1/height',
    'photos/1/id',
    'photos/1/image',
    'photos/1/width',
    'photos/2/height',
    'photos/2/id',
    'photos/2/image',
    'photos/2/width',
    'photos/3/height',
    'photos/3/id',
    'photos/3/image',
    'photos/3/width',
    'photos/4/height',
    'photos/4/id',
    'photos/4/image',
    'photos/4/width',
    'photos/5/height',
    'photos/5/id',
    'photos/5/image',
    'photos/5/width',
    'photos/6/height',
    'photos/6/id',
    'photos/6/image',
    'photos/6/width',
    'photos/7/height',
    'photos/7/id',
    'photos/7/image',
    'photos/7/width',
    'photos/8/height',
    'photos/8/id',
    'photos/8/image',
    'photos/8/width',
    'photos/9/height',
    'photos/9/id',
    'photos/9/image',
    'photos/9/width',
    'photos/10/height',
    'photos/10/id',
    'photos/10/image',
    'photos/10/width',
    'photos/11/height',
    'photos/11/id',
    'photos/11/image',
    'photos/11/width',
    'photos/12/height',
    'photos/12/id',
    'photos/12/image',
    'photos/12/width',
    'photos/13/height',
    'photos/13/id',
    'photos/13/image',
    'photos/13/width',
    'photos/14/height',
    'photos/14/id',
    'photos/14/image',
    'photos/14/width',
    'photos/15/height',
    'photos/15/id',
    'photos/15/image',
    'photos/15/width',
    'photos/16/height',
    'photos/16/id',
    'photos/16/image',
    'photos/16/width',
    'photos/17/height',
    'photos/17/id',
    'photos/17/image',
    'photos/17/width',
    'photos/18/height',
    'photos/18/id',
    'photos/18/image',
    'photos/18/width',
    'photos/19/height',
    'photos/19/id',
    'photos/19/image',
    'photos/19/width',
    'photos/20/height',
    'photos/20/id',
    'photos/20/image',
    'photos/20/width',
    'photos/21/height',
    'photos/21/id',
    'photos/21/image',
    'photos/21/width',
    'photos/22/height',
    'photos/22/id',
    'photos/22/image',
    'photos/22/width',
    'photos/23/height',
    'photos/23/id',
    'photos/23/image',
    'photos/23/width',
    'photos/24/height',
    'photos/24/id',
    'photos/24/image',
    'photos/24/width',
    'photos/25/height',
    'photos/25/id',
    'photos/25/image',
    'photos/25/width',
    'photos/26/height',
    'photos/26/id',
    'photos/26/image',
    'photos/26/width',
    'photos/27/height',
    'photos/27/id',
    'photos/27/image',
    'photos/27/width',
    'photos/28/height',
    'photos/28/id',
    'photos/28/image',
    'photos/28/width',

    'photos/29/height',
    'photos/29/id',
    'photos/29/image',
    'photos/29/width',
    'photos/30/height',
    'photos/30/id',
    'photos/30/image',
    'photos/30/width',
    'photos/31/height',
    'photos/31/id',
    'photos/31/image',
    'photos/31/width',
    'photos/32/height',
    'photos/32/id',
    'photos/32/image',
    'photos/32/width',
    'photos/33/height',
    'photos/33/id',
    'photos/33/image',
    'photos/33/width',
    'photos/34/height',
    'photos/34/id',
    'photos/34/image',
    'photos/34/width',
    'photos/35/height',
    'photos/35/id',
    'photos/35/image',
    'photos/35/width',

    'photos/36/height',
    'photos/36/id',
    'photos/36/image',
    'photos/36/width',
    'photos/37/height',
    'photos/37/id',
    'photos/37/image',
    'photos/37/width',
    'photos/38/height',
    'photos/38/id',
    'photos/38/image',
    'photos/38/width',
    'photos/39/height',
    'photos/39/id',
    'photos/39/image',
    'photos/39/width',
    'photos/40/height',
    'photos/40/id',
    'photos/40/image',
    'photos/40/width',
    'photos/41/height',
    'photos/41/id',
    'photos/41/image',
    'photos/41/width',

    'photos/42/height',
    'photos/42/id',
    'photos/42/image',
    'photos/42/width',
    'photos/43/height',
    'photos/43/id',
    'photos/43/image',
    'photos/43/width',
    'photos/44/height',
    'photos/44/id',
    'photos/44/image',
    'photos/44/width',
    'photos/45/height',
    'photos/45/id',
    'photos/45/image',
    'photos/45/width',

    'photos/46/height',
    'photos/46/id',
    'photos/46/image',
    'photos/46/width',
    'photos/47/height',
    'photos/47/id',
    'photos/47/image',
    'photos/47/width',
    'photos/48/height',
    'photos/48/id',
    'photos/48/image',
    'photos/48/width',
    'photos/49/height',
    'photos/49/id',
    'photos/49/image',
    'photos/49/width',
    'photos/50/height',
    'photos/50/id',
    'photos/50/image',
    'photos/50/width',

    'photos/51/height',
    'photos/51/id',
    'photos/51/image',
    'photos/51/width',
    'photos/52/height',
    'photos/52/id',
    'photos/52/image',
    'photos/52/width',
    'photos/53/height',
    'photos/53/id',
    'photos/53/image',
    'photos/53/width',
    'photos/54/height',
    'photos/54/id',
    'photos/54/image',
    'photos/54/width',
    'photos/55/height',
    'photos/55/id',
    'photos/55/image',
    'photos/55/width',
    'photos/56/height',
    'photos/56/id',
    'photos/56/image',
    'photos/56/width',

    'photos/57/height',
    'photos/57/id',
    'photos/57/image',
    'photos/57/width',
    'photos/58/height',
    'photos/58/id',
    'photos/58/image',
    'photos/58/width',
    'photos/59/height',
    'photos/59/id',
    'photos/59/image',
    'photos/59/width',


    'placeInfo/address',
    'placeInfo/addressObj/city',
    'placeInfo/addressObj/country',
    'placeInfo/addressObj/postalcode',
    'placeInfo/addressObj/state',
    'placeInfo/addressObj/street1',
    'placeInfo/addressObj/street2',
    'placeInfo/id',
    'placeInfo/latitude',
    'placeInfo/locationString',
    'placeInfo/longitude',
    'placeInfo/numberOfReviews',
    'placeInfo/ratingHistogram/count1',
    'placeInfo/ratingHistogram/count2',
    'placeInfo/ratingHistogram/count3',
    'placeInfo/ratingHistogram/count4',
    'placeInfo/ratingHistogram/count5',
    'placeInfo/webUrl',
    'placeInfo/website',
    'subratings/0/name',
    'subratings/0/value',
    'subratings/1/name',
    'subratings/1/value',
    'subratings/2/name',
    'subratings/2/value',
    'subratings/3/name',
    'subratings/3/value',
    'subratings/4/name',
    'subratings/4/value',
    'subratings/5/name',
    'subratings/5/value',
    'travelDate',
    'travelDate',
    'url',

    'user',
    'user/avatar',
    'user/avatar/height',
    'user/avatar/id',
    'user/avatar/image',
    'user/avatar/width',
    'user/contributions/helpfulVotes',
    'user/link',
    'user/userId',
    'user/userLocation',
    'user/userLocation/id',
    'user/username'

]
CinnWild_Yala_TA = CinnWild_Yala_TA.drop(columns=columns_to_drop, errors='ignore')

In [ ]:
CinnWild_Yala_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 23 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   lang                                   2499 non-null   object 
 1   ownerResponse/publishedDate            2246 non-null   object 
 2   ownerResponse/text                     2246 non-null   object 
 3   placeInfo/name                         2499 non-null   object 
 4   placeInfo/rating                       2499 non-null   float64
 5   publishedDate                          2499 non-null   object 
 6   publishedPlatform                      2499 non-null   object 
 7   rating                                 2499 non-null   int64  
 8   roomTip                                350 non-null    object 
 9   text                                   2499 non-null   object 
 10  title                                  2494 non-null   object 
 11  trip

In [ ]:
# Re-naming/Standardizing the columns
CinnWild_Yala_TA = CinnWild_Yala_TA.rename(columns={
    'lang': 'Language',
    'ownerResponse/publishedDate': 'OwnerResponsepublishedDate',
    'ownerResponse/text': 'OwnerResponse',
    'placeInfo/name': 'Hotel/Resort',
    'placeInfo/rating ': 'AverageRating',
    'publishedDate': 'RatingPublishedDate',
    'publishedPlatform': 'RatingPublishedPlatform',
    'rating': 'HotelRating',
    'roomTip': 'RoomtipText',
    'text': 'ReviewText',
    'title': 'ReviewTitle',
    'tripType': 'TripType',
    'user/contributions/totalContributions': 'TotalReviewsbyUser',
    'user/name': 'UserName',
    'user/userLocation/name': 'UserLocation',
    'user/userLocation/shortName': 'UserLocationinShort',
    'platform': 'Platform'

})


In [ ]:
CinnWild_Yala_TA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Language                    2499 non-null   object 
 1   OwnerResponsepublishedDate  2246 non-null   object 
 2   OwnerResponse               2246 non-null   object 
 3   Hotel/Resort                2499 non-null   object 
 4   placeInfo/rating            2499 non-null   float64
 5   RatingPublishedDate         2499 non-null   object 
 6   RatingPublishedPlatform     2499 non-null   object 
 7   HotelRating                 2499 non-null   int64  
 8   RoomtipText                 350 non-null    object 
 9   ReviewText                  2499 non-null   object 
 10  ReviewTitle                 2494 non-null   object 
 11  TripType                    2499 non-null   object 
 12  TotalReviewsbyUser          2497 non-null   float64
 13  UserName                    2497 

In [ ]:
display(CinnWild_Yala_TA.head(10))

,Language,OwnerResponsepublishedDate,OwnerResponse,Hotel/Resort,placeInfo/rating,RatingPublishedDate,RatingPublishedPlatform,HotelRating,RoomtipText,ReviewText,...,UserName,UserLocation,UserLocationinShort,CleanlinessRating,LocationRating,ComfortRating,StaffRating,SleepQualityRating,ValueforMoneyRating,Platform
0,en,NaN,NaN,Cinnamon Wild Yala,4.6,2025-12-10,OTHER,5,NaN,"Great landscape, great hospitality, \nGreat ho...",...,Road24374295700,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.0,TripAdvisor
1,en,2024-07-28,"Dear Chamila,\n\nGreetings from Cinnamon Wild ...",Cinnamon Wild Yala,4.6,2024-07-12,OTHER,5,NaN,Fantastic stay with lots of animal visitors .C...,...,chamilaherath78,"Doha, Qatar",Doha,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
2,en,NaN,NaN,Cinnamon Wild Yala,4.6,2025-12-10,OTHER,5,NaN,A truly unforgettable stay at Cinnamon Wild Ya...,...,Stavros K,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
3,en,NaN,NaN,Cinnamon Wild Yala,4.6,2025-12-09,OTHER,5,NaN,We loved every aspect of our stay at Cinnamon ...,...,David M,NaN,NaN,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
4,en,NaN,NaN,Cinnamon Wild Yala,4.6,2025-12-09,OTHER,5,NaN,Great location in the heart of the national pa...,...,jacquivm1975,"Pennant Hills, Australia",Pennant Hills,5.0,5.0,5.0,5.0,5.0,5.0,TripAdvisor
5,en,NaN,NaN,Cinnamon Wild Yala,4.6,2025-12-08,MOBILE,5,"Engage with the staff, they are beautiful warm...","A fantastic hotel by a lake full of Crocs, wat...",...,Dezmondo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
6,en,2025-12-08,"Dear Guest,\n\nGreetings from Cinnamon Wild Ya...",Cinnamon Wild Yala,4.6,2025-12-07,OTHER,5,Rooms are little small in size for this level ...,Great location. Excellent hospitality. Rooms a...,...,Inspire599275,NaN,NaN,5.0,5.0,5.0,5.0,5.0,4.0,TripAdvisor
7,en,2025-12-08,"Dear Joyce,\n\nGreetings from Cinnamon Wild Ya...",Cinnamon Wild Yala,4.6,2025-12-07,OTHER,5,NaN,Umesh and his team services were very good. Ve...,...,MummyJoyce,"Singapore, Singapore",Singapore,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor
8,en,2025-12-08,"Dear Promero,\n\nGreetings from Cinnamon Wild ...",Cinnamon Wild Yala,4.6,2025-12-07,OTHER,5,NaN,"Beautiful place, amazing service specially fro...",...,pedro R,"Huelva, Spain",Huelva,5.0,5.0,4.0,5.0,4.0,4.0,TripAdvisor
9,en,2025-12-08,"Dear Samantha,\n\nGreetings from Cinnamon Wild...",Cinnamon Wild Yala,4.6,2025-12-07,OTHER,5,NaN,We were truly impressed with the service provi...,...,samantha k,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TripAdvisor


In [ ]:
# Save the cleaned DataFrame as Excel
excel_filename = 'CinnWild_Yala_TA_Cleaned.xlsx'

CinnWild_Yala_TA.to_excel(excel_filename, index=False)

print(f"Dataset saved as {excel_filename}")
print("Download will start automatically...")

# Trigger download in Colab
from google.colab import files
files.download(excel_filename)


Dataset saved as CinnWild_Yala_TA_Cleaned.xlsx
Download will start automatically...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>